consumer airfare report:
https://data.transportation.gov/Aviation/Consumer-Airfare-Report-Table-2-Top-1-000-City-Pai/wqw2-rjgd/about_data

on time monthly data:
https://www.transtats.bts.gov/Fields.asp?gnoyr_VQ=FGK

airline codes:
https://www.bansard.com/sites/default/files/download_documents/Bansard-airlines-codes-IATA-ICAO.xlsx&ved=2ahUKEwj2zPThgpSIAxWLITQIHdmMGsEQFnoECAwQAQ&usg=AOvVaw1t6UbPkojIZrVMuOTDghg9

In [14]:
import sys
import os

# Get the project root directory
PROJECT_ROOT = os.path.abspath(os.path.join(os.pardir))

# Add the project root to sys.path
sys.path.append(PROJECT_ROOT)

# Now you can import the config module
from config.config import DATA_PATH

In [15]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from urllib.request import urlretrieve


In [16]:
pd.set_option('display.max_columns', None)

# Import Flight Performance Data

In [17]:
import pandas as pd
import os

def read_and_combine_csv(base_path, data_folder):
    """
    Reads multiple CSV files from a specified directory and combines them into a single DataFrame.

    Args:
    base_path (str): The base directory path where data folders are stored.
    data_folder (str): The folder name containing the CSV files to read.

    Returns:
    DataFrame: A combined DataFrame containing data from all CSV files in the specified directory.
    """
    # Construct the full path to the folder containing CSV files
    folder_path = os.path.join(base_path, data_folder)
    
    # Initialize an empty list to store individual DataFrames
    dataframes_list = []
    
    # Iterate through each file in the directory
    for filename in os.listdir(folder_path):
        if filename.endswith('.csv'):
            file_path = os.path.join(folder_path, filename)
            df = pd.read_csv(file_path, low_memory=False)
            dataframes_list.append(df)
    
    # Concatenate all DataFrames in the list into one DataFrame
    combined_df = pd.concat(dataframes_list, ignore_index=True)
    
    return combined_df

In [18]:
# Assume DATA_PATH is already imported from your config
flight_data = '2023-performance-data'
df_2023 = read_and_combine_csv(DATA_PATH+'/raw', flight_data)

# Check the combined DataFrame
print(df_2023.head())

   Year  Quarter  Month  DayofMonth  DayOfWeek  FlightDate  \
0  2023        3      8           6          7  2023-08-06   
1  2023        3      8           7          1  2023-08-07   
2  2023        3      8           9          3  2023-08-09   
3  2023        3      8          10          4  2023-08-10   
4  2023        3      8          12          6  2023-08-12   

  Marketing_Airline_Network Operated_or_Branded_Code_Share_Partners  \
0                        NK                                      NK   
1                        NK                                      NK   
2                        NK                                      NK   
3                        NK                                      NK   
4                        NK                                      NK   

   DOT_ID_Marketing_Airline IATA_Code_Marketing_Airline  \
0                     20416                          NK   
1                     20416                          NK   
2                     2

In [19]:
df_codes = pd.read_csv(DATA_PATH + '/raw/L_AIRLINE_ID.csv')

## Inspect Data Quality

In [20]:
df_2023.shape

(7278739, 120)

In [21]:
df_2023.head()

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Marketing_Airline_Network,Operated_or_Branded_Code_Share_Partners,DOT_ID_Marketing_Airline,IATA_Code_Marketing_Airline,Flight_Number_Marketing_Airline,Originally_Scheduled_Code_Share_Airline,DOT_ID_Originally_Scheduled_Code_Share_Airline,IATA_Code_Originally_Scheduled_Code_Share_Airline,Flight_Num_Originally_Scheduled_Code_Share_Airline,Operating_Airline,DOT_ID_Operating_Airline,IATA_Code_Operating_Airline,Tail_Number,Flight_Number_Operating_Airline,OriginAirportID,OriginAirportSeqID,OriginCityMarketID,Origin,OriginCityName,OriginState,OriginStateFips,OriginStateName,OriginWac,DestAirportID,DestAirportSeqID,DestCityMarketID,Dest,DestCityName,DestState,DestStateFips,DestStateName,DestWac,CRSDepTime,DepTime,DepDelay,DepDelayMinutes,DepDel15,DepartureDelayGroups,DepTimeBlk,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrTime,ArrDelay,ArrDelayMinutes,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,Cancelled,CancellationCode,Diverted,CRSElapsedTime,ActualElapsedTime,AirTime,Flights,Distance,DistanceGroup,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,FirstDepTime,TotalAddGTime,LongestAddGTime,DivAirportLandings,DivReachedDest,DivActualElapsedTime,DivArrDelay,DivDistance,Div1Airport,Div1AirportID,Div1AirportSeqID,Div1WheelsOn,Div1TotalGTime,Div1LongestGTime,Div1WheelsOff,Div1TailNum,Div2Airport,Div2AirportID,Div2AirportSeqID,Div2WheelsOn,Div2TotalGTime,Div2LongestGTime,Div2WheelsOff,Div2TailNum,Div3Airport,Div3AirportID,Div3AirportSeqID,Div3WheelsOn,Div3TotalGTime,Div3LongestGTime,Div3WheelsOff,Div3TailNum,Div4Airport,Div4AirportID,Div4AirportSeqID,Div4WheelsOn,Div4TotalGTime,Div4LongestGTime,Div4WheelsOff,Div4TailNum,Div5Airport,Div5AirportID,Div5AirportSeqID,Div5WheelsOn,Div5TotalGTime,Div5LongestGTime,Div5WheelsOff,Div5TailNum,Duplicate,Unnamed: 119
0,2023,3,8,6,7,2023-08-06,NK,NK,20416,NK,2252,NaN,NaN,NaN,NaN,NK,20416,NK,N978NK,2252,12889,1288904,32211,LAS,"Las Vegas, NV",NV,32,Nevada,85,11057,1105703,31057,CLT,"Charlotte, NC",NC,37,North Carolina,36,2359,NaN,NaN,NaN,NaN,NaN,2300-2359,NaN,NaN,NaN,NaN,722,NaN,NaN,NaN,NaN,NaN,0700-0759,1.0,B,0.0,263.0,NaN,NaN,1.0,1916.0,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
1,2023,3,8,7,1,2023-08-07,NK,NK,20416,NK,2252,NaN,NaN,NaN,NaN,NK,20416,NK,N974NK,2252,12889,1288904,32211,LAS,"Las Vegas, NV",NV,32,Nevada,85,11057,1105703,31057,CLT,"Charlotte, NC",NC,37,North Carolina,36,2359,115.0,76.0,76.0,1.0,5.0,2300-2359,19.0,134.0,822.0,9.0,722,831.0,69.0,69.0,1.0,4.0,0700-0759,0.0,NaN,0.0,263.0,256.0,228.0,1.0,1916.0,8,3.0,0.0,1.0,0.0,65.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
2,2023,3,8,9,3,2023-08-09,NK,NK,20416,NK,2252,NaN,NaN,NaN,NaN,NK,20416,NK,N519NK,2252,12889,1288904,32211,LAS,"Las Vegas, NV",NV,32,Nevada,85,11057,1105703,31057,CLT,"Charlotte, NC",NC,37,North Carolina,36,2310,2259.0,-11.0,0.0,0.0,-1.0,2300-2359,14.0,2313.0,605.0,10.0,628,615.0,-13.0,0.0,0.0,-1.0,0600-0659,0.0,NaN,0.0,258.0,256.0,232.0,1.0,1916.0,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
3,2023,3,8,10,4,2023-08-10,NK,NK,20416,NK,2252,NaN,NaN,NaN,NaN,NK,20416,NK,N532NK,2252,12889,1288904,32211,LAS,"Las Vegas, NV",NV,32,Nevada,85,11057,1105703,31057,CLT,"Charlotte, NC",NC,37,North Carolina,36,2310,2302.0,-8.0,0.0,0.0,-1.0,2300-2359,12.0,2314.0,551.0,7.0,628,558.0,-30.0,0.0,0.0,-2.0,0600-0659,0.0,NaN,0.0,258.0,236.0,217.0,1.0,1916.0,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [22]:
df_2023.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7278739 entries, 0 to 7278738
Columns: 120 entries, Year to Unnamed: 119
dtypes: float64(71), int64(22), object(27)
memory usage: 6.5+ GB


In [23]:
df_2023.columns = df_2023.columns.str.strip()

## Missing Values

In [24]:
null_vals = df_2023.isna().sum().reset_index()
null_vals.columns = ['column', 'num_missing']
null_vals['perc_missing'] = round(null_vals['num_missing'] / len(df_2023) *100, 3)
missing_over_50 = null_vals[null_vals['perc_missing'] > 50].sort_values('perc_missing', ascending=False)
print(missing_over_50.shape)
missing_over_50

(58, 3)


,column,num_missing,perc_missing
119,Unnamed: 119,7278739,100.000
106,Div4TotalGTime,7278739,100.000
94,Div3Airport,7278737,100.000
95,Div3AirportID,7278737,100.000
96,Div3AirportSeqID,7278737,100.000
97,Div3WheelsOn,7278737,100.000
98,Div3TotalGTime,7278737,100.000
99,Div3LongestGTime,7278737,100.000
100,Div3WheelsOff,7278738,100.000
101,Div3TailNum,7278738,100.000


Fifty-eight columns are missing over 50% of their values, 53 of which are missing over 90% of their values. Most of these columns are dealing with details about diverted flights. There is a column, `Diverted`, which captures whether or not a flight was diverted and it does not contain any missing values. At this point, it will be best to remove the columns with over 90% of their values missing. 

The different delay types (`LateAircraftDelay`, `SecurityDelay`, `NASDelay`, `WeatherDelay`, `CarrierDelay`) may come in handy as I would expect most flights do not experience delays. These different types of delays may inform some segmentation of delayed flights later on. 

In [62]:
# drop columns with over 80% missing values
cols_to_drop = null_vals[null_vals['perc_missing'] >= 80]['column'].tolist()

df = df_2023.drop(columns=cols_to_drop)

In [63]:
missing_under_50 = null_vals[(null_vals['perc_missing'] <= 50) & (null_vals['perc_missing'] > 0)].sort_values('perc_missing', ascending=False)
print(missing_under_50.shape)
missing_under_50


(17, 3)


,column,num_missing,perc_missing
61,AirTime,111695,1.535
60,ActualElapsedTime,111695,1.535
54,ArrivalDelayGroups,111695,1.535
53,ArrDel15,111695,1.535
52,ArrDelayMinutes,111695,1.535
51,ArrDelay,111695,1.535
48,TaxiIn,95647,1.314
50,ArrTime,95646,1.314
47,WheelsOn,95647,1.314
46,WheelsOff,93119,1.279


Seventeen columns have less than 2% of their values missing. 

Earlier I deleted rows with missing values because of the % being insignificant. However, some of these missing observations likely overlap with the 11,032 cancelled flights. Others are just missing informaiton for some reason. I found this out because when I wanted to see the counts of cancelled vs not cancelled flights, I only saw details for not cancelled flights. 

It would make sense that cancelled flights would have missing information in other columns. I want to identify which flights were cancelled and how many there were. 

In [64]:
df['Cancelled'].value_counts()

Cancelled
0.0    7184842
1.0      93897
Name: count, dtype: int64

In [65]:
cancelled_flights = df['Cancelled'] == 1.0
print('Total number of cancelled flights: ', len(df[cancelled_flights]))

Total number of cancelled flights:  93897


In [66]:
not_cancelled = df[~cancelled_flights]
not_cancelled_missing = not_cancelled.isna().sum().reset_index()
not_cancelled_missing.columns = ['Column', 'Count_Missing']
not_cancelled_missing[not_cancelled_missing['Count_Missing'] > 0]

,Column,Count_Missing
43,WheelsOn,1750
44,TaxiIn,1750
46,ArrTime,1749
47,ArrDelay,17798
48,ArrDelayMinutes,17798
49,ArrDel15,17798
50,ArrivalDelayGroups,17798
54,CRSElapsedTime,1
55,ActualElapsedTime,17798
56,AirTime,17798


There were over 93,000 cancelled flights in 2023. However, some of the data is still missing information for several columns even after excluding the cancelled flights. 

I still maintain that the different delay types (`LateAircraftDelay`, `SecurityDelay`, `NASDelay`, `WeatherDelay`, `CarrierDelay`) may come in handy as I would expect most flights do not experience delays.

However, I believe I should remove observations that are missing data in the other columns. 

In [67]:
missing_critical_data = ((df['Cancelled'] == 0) & (df['ArrTime'].isna())) | ((df['Cancelled'] == 0) & (df['ArrDelayMinutes'].isna()))
print('Total non-cancelled flights with missing data: ', missing_critical_data.sum())

Total non-cancelled flights with missing data:  17798


In [68]:
# drop rows missing less than 3% of values
df = df[~missing_critical_data]

In [69]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7260941 entries, 0 to 7278738
Data columns (total 67 columns):
 #   Column                                   Dtype  
---  ------                                   -----  
 0   Year                                     int64  
 1   Quarter                                  int64  
 2   Month                                    int64  
 3   DayofMonth                               int64  
 4   DayOfWeek                                int64  
 5   FlightDate                               object 
 6   Marketing_Airline_Network                object 
 7   Operated_or_Branded_Code_Share_Partners  object 
 8   DOT_ID_Marketing_Airline                 int64  
 9   IATA_Code_Marketing_Airline              object 
 10  Flight_Number_Marketing_Airline          int64  
 11  Operating_Airline                        object 
 12  DOT_ID_Operating_Airline                 int64  
 13  IATA_Code_Operating_Airline              object 
 14  Tail_Number            

## Data Types

The `FlightDate` column will need to be converted to a datetime object. I'll be able to see if there are any inconsistencies with the range of dates. I expect only flight details for January 2023.

In [70]:
# convert FlightDate to datetime object
df['FlightDate'] = pd.to_datetime(df['FlightDate'], errors='coerce')
df['FlightDate'].info()

<class 'pandas.core.series.Series'>
Index: 7260941 entries, 0 to 7278738
Series name: FlightDate
Non-Null Count    Dtype         
--------------    -----         
7260941 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 110.8 MB


In [71]:
df['FlightDate'].describe()

count                          7260941
mean     2023-07-03 18:25:02.977754112
min                2023-01-01 00:00:00
25%                2023-04-05 00:00:00
50%                2023-07-05 00:00:00
75%                2023-10-02 00:00:00
max                2023-12-31 00:00:00
Name: FlightDate, dtype: object

In [72]:
df['FlightDate_date_only'] = df['FlightDate'].dt.date
df['FlightDate_date_only'].describe()

count        7260941
unique           365
top       2023-11-26
freq           21864
Name: FlightDate_date_only, dtype: object

The range of flight dates is as expected, ranging from January 1st to December 31st. 

The most common date for flights was November 26th with over 21,864 scheduled flights. This is not surprising as it is Thanksgiving weekend, a holiday. It will likely be useful to add a column for whether a travel date was on a holday. 

## Redundant Columns with Flight Operations

The first few rows of the dataframe seemed to have similar information for `Marketing_Airline_Network`, `Operated_or_Branded_Code_Share_Partners`, and `IATA_Code_Marketing_Airline`. There also appear to be identical data for the `Operating_Airline` and `IATA_Code_Operating_Airline` columns. I want to take a closer look at these columns. 

In [73]:
df[['Marketing_Airline_Network', 'Operated_or_Branded_Code_Share_Partners', 'IATA_Code_Marketing_Airline', 'Operating_Airline', 'IATA_Code_Operating_Airline']]

,Marketing_Airline_Network,Operated_or_Branded_Code_Share_Partners,IATA_Code_Marketing_Airline,Operating_Airline,IATA_Code_Operating_Airline
0,NK,NK,NK,NK,NK
1,NK,NK,NK,NK,NK
2,NK,NK,NK,NK,NK
3,NK,NK,NK,NK,NK
4,NK,NK,NK,NK,NK
...,...,...,...,...,...
7278734,AA,AA_CODESHARE,AA,OH,OH
7278735,AA,AA_CODESHARE,AA,OH,OH
7278736,AA,AA_CODESHARE,AA,OH,OH
7278737,AA,AA_CODESHARE,AA,OH,OH


After reading documentation, the Marketing Airline Network and/or IATA Code is not always unique as it can be assigned to different airlines over time. Also, these codes may not always be unique. The identify unique airline carriers, the `DOT_ID_Marketing_Airline` column should be used as this is unique to the airline carrier and is defined as one holding and reporting under the same DOT certificate regardless of its Code, Name, or holding company/corporation.

The `Operated_or_Branded_Code_Share_Partners` column identifies whether the flight was operate by one of the airline's code share partners. A codeshare flight is an agreement between airlines to sell seats on each other’s flights. This gives the appearance of airlines flying to more destinations. By doing so, the airlines typically share the revenue on that ticket. The `DOT_ID_Operating_Airline` column should be used to identify which airline carrier is oeprating the flight. 

Creating a new column denoting whether a flight was a code share flight will potentially allow for segregating flights based on differing operations. 

In [74]:
# Compare the two columns to find where they are different
code_share_flights = (df['Marketing_Airline_Network'] != df['Operated_or_Branded_Code_Share_Partners'])
print('The total number of code share flights: ', len(df[code_share_flights]))
print('The percent of flights that are code share flights: ', round(len(df[code_share_flights])/len(df) * 100, 2), '%')


The total number of code share flights:  2019136
The percent of flights that are code share flights:  27.81 %


Over a quarter of all flights in 2023 were code share flights. This seems like a significant proportion. 

In [75]:
df['Code_Share_Flight'] = code_share_flights
df['Code_Share_Flight'] = df['Code_Share_Flight'].astype(int)

In [76]:
df['Code_Share_Flight'].head()

0    0
1    0
2    0
3    0
4    0
Name: Code_Share_Flight, dtype: int64

I will drop the `Operated_or_Branded_Code_Share_Partners` column now that I have capture whether or not a flight was a code share flight. 

In [77]:
ops_columns_to_drop = ['Operated_or_Branded_Code_Share_Partners']

In [78]:
# Compare the two columns to find where they are different
differences = df['Marketing_Airline_Network'] != df['IATA_Code_Marketing_Airline']

# Count the number of differences
count_differences = differences.sum()

print(f"Number of differences between the two columns: {count_differences}")


Number of differences between the two columns: 0


Since Marketing Airline Network Code and the IATA Code don't have any differences, keeping both columns is unnecessary. I will drop the `Marketing_Airline_Network`column.

In [79]:
ops_columns_to_drop.append('Marketing_Airline_Network')

In [80]:
# Compare the two columns to find where they are different
differences = df['Flight_Number_Marketing_Airline'] != df['Flight_Number_Operating_Airline']

# Count the number of differences
count_differences = differences.sum()

print(f"Number of differences between the two columns: {count_differences}")


Number of differences between the two columns: 575


The flight numbers are identical for nearly all observations except for 575 flights. I will only keep the `Flight_Number_Marketing_Airline` column.

In [81]:
ops_columns_to_drop.append('Flight_Number_Operating_Airline')

In [82]:
# Compare the two columns to find where they are different
differences = df['Operating_Airline'] != df['IATA_Code_Operating_Airline']

# Count the number of differences
count_differences = differences.sum()

print(f"Number of differences between the two columns: {count_differences}")


Number of differences between the two columns: 0


Since Operating Airline Code and the IATA Code don't have any differences, keeping both columns is unnecessary. I will drop the `Operating_Airline`column.

In [83]:
ops_columns_to_drop.append('Operating_Airline')

In [84]:
df = df.drop(columns=ops_columns_to_drop)

In [85]:
df.head()

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,DOT_ID_Marketing_Airline,IATA_Code_Marketing_Airline,Flight_Number_Marketing_Airline,DOT_ID_Operating_Airline,IATA_Code_Operating_Airline,Tail_Number,OriginAirportID,OriginAirportSeqID,OriginCityMarketID,Origin,OriginCityName,OriginState,OriginStateFips,OriginStateName,OriginWac,DestAirportID,DestAirportSeqID,DestCityMarketID,Dest,DestCityName,DestState,DestStateFips,DestStateName,DestWac,CRSDepTime,DepTime,DepDelay,DepDelayMinutes,DepDel15,DepartureDelayGroups,DepTimeBlk,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrTime,ArrDelay,ArrDelayMinutes,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,Cancelled,Diverted,CRSElapsedTime,ActualElapsedTime,AirTime,Flights,Distance,DistanceGroup,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,DivAirportLandings,Duplicate,FlightDate_date_only,Code_Share_Flight
0,2023,3,8,6,7,2023-08-06,20416,NK,2252,20416,NK,N978NK,12889,1288904,32211,LAS,"Las Vegas, NV",NV,32,Nevada,85,11057,1105703,31057,CLT,"Charlotte, NC",NC,37,North Carolina,36,2359,NaN,NaN,NaN,NaN,NaN,2300-2359,NaN,NaN,NaN,NaN,722,NaN,NaN,NaN,NaN,NaN,0700-0759,1.0,0.0,263.0,NaN,NaN,1.0,1916.0,8,NaN,NaN,NaN,NaN,NaN,0.0,N,2023-08-06,0
1,2023,3,8,7,1,2023-08-07,20416,NK,2252,20416,NK,N974NK,12889,1288904,32211,LAS,"Las Vegas, NV",NV,32,Nevada,85,11057,1105703,31057,CLT,"Charlotte, NC",NC,37,North Carolina,36,2359,115.0,76.0,76.0,1.0,5.0,2300-2359,19.0,134.0,822.0,9.0,722,831.0,69.0,69.0,1.0,4.0,0700-0759,0.0,0.0,263.0,256.0,228.0,1.0,1916.0,8,3.0,0.0,1.0,0.0,65.0,0.0,N,2023-08-07,0
2,2023,3,8,9,3,2023-08-09,20416,NK,2252,20416,NK,N519NK,12889,1288904,32211,LAS,"Las Vegas, NV",NV,32,Nevada,85,11057,1105703,31057,CLT,"Charlotte, NC",NC,37,North Carolina,36,2310,2259.0,-11.0,0.0,0.0,-1.0,2300-2359,14.0,2313.0,605.0,10.0,628,615.0,-13.0,0.0,0.0,-1.0,0600-0659,0.0,0.0,258.0,256.0,232.0,1.0,1916.0,8,NaN,NaN,NaN,NaN,NaN,0.0,N,2023-08-09,0
3,2023,3,8,10,4,2023-08-10,20416,NK,2252,20416,NK,N532NK,12889,1288904,32211,LAS,"Las Vegas, NV",NV,32,Nevada,85,11057,1105703,31057,CLT,"Charlotte, NC",NC,37,North Carolina,36,2310,2302.0,-8.0,0.0,0.0,-1.0,2300-2359,12.0,2314.0,551.0,7.0,628,558.0,-30.0,0.0,0.0,-2.0,0600-0659,0.0,0.0,258.0,236.0,217.0,1.0,1916.0,8,NaN,NaN,NaN,NaN,NaN,0.0,N,2023-08-10,0
4,2023,3,8,12,6,2023-08-12,20416,NK,2252,20416,NK,N529NK,12889,1288904,32211,LAS,"Las Vegas, NV",NV,32,Nevada,85,11057,1105703,31057,CLT,"Charlotte, NC",NC,37,North Carolina,36,2310,2314.0,4.0,4.0,0.0,0.0,2300-2359,18.0,2332.0,618.0,6.0,628,624.0,-4.0,0.0,0.0,-1.0,0600-0659,0.0,0.0,258.0,250.0,226.0,1.0,1916.0,8,NaN,NaN,NaN,NaN,NaN,0.0,N,2023-08-12,0


In [86]:
airport_details = df[['OriginAirportID', 'OriginAirportSeqID', 'OriginCityMarketID', 'Origin', 'OriginCityName', 'OriginState', 'OriginStateFips', 'OriginStateName']].drop_duplicates()
airport_details = airport_details.rename(columns={
    'OriginAirportID':'Airport_ID',
    'OriginAirportSeqID':'Airport_Seq_ID',
    'Origin_City_Market_ID':'City_Market_ID',
    'Origin':'Airport_Code',
    'OriginCityName':'City_Name',
    'OriginState':'State_Code',
    'OriginStateFips':'State_Fips', 
    'OriginStateName':'State_Name'
})

In [87]:
airport_details

,Airport_ID,Airport_Seq_ID,OriginCityMarketID,Airport_Code,City_Name,State_Code,State_Fips,State_Name
0,12889,1288904,32211,LAS,"Las Vegas, NV",NV,32,Nevada
24,11697,1169706,32467,FLL,"Fort Lauderdale, FL",FL,12,Florida
55,13204,1320402,31454,MCO,"Orlando, FL",FL,12,Florida
77,10397,1039707,30397,ATL,"Atlanta, GA",GA,13,Georgia
120,10423,1042302,30423,AUS,"Austin, TX",TX,48,Texas
...,...,...,...,...,...,...,...,...
3872263,11315,1131503,31315,DIK,"Dickinson, ND",ND,38,North Dakota
3888468,13256,1325602,33256,MFE,"Mission/McAllen/Edinburg, TX",TX,48,Texas
4187478,13983,1398305,33983,OWB,"Owensboro, KY",KY,21,Kentucky
4294964,13459,1345903,33459,MQT,"Marquette, MI",MI,26,Michigan


## Redundant Columns with Origin and Destination Details


`OriginAirportID` and `OriginAirportSeqID` both provide identification numbers used by the DOT to identify a unique airport. The additional numbers in the `OriginAirportSeqID` column provide information about the airport at a given point in time. Since both of these point to the same unique airport, we only need one of the columns. Similarly, `DestAirportID` and `DestAirportSeqID` provide information about the unique destination airport. We only need one of the columns. I will drop both columns with SeqID information. 

In [88]:
location_details_to_drop = ['OriginAirportSeqID', 'DestAirportSeqID']

The `OriginCityName` and `DestCityName` include the state or country names after the comma. Since this information is already tracked in the `OriginState` column, we don't also need state informatino in the city column.

In [89]:
# Split the 'OriginCityName' into two columns but handle cases where there might not be a comma
df['Origin_City'] = df['OriginCityName'].str.split(',').str[0]
df['PotentialState'] = df['OriginCityName'].str.split(',').str[1]

# Fill NaNs in 'PotentialState' with an empty string or a placeholder if no comma was present
df['PotentialState'] = df['PotentialState'].fillna('')

# Trim any whitespace that might be around the state names
df['PotentialState'] = df['PotentialState'].str.strip()

# Proceed with your state comparison and adjustment
# This assumes you have an 'OriginState' column to compare with
df['CheckStateMatch'] = df['PotentialState'] == df['OriginState']
df['Origin_City'] = df.apply(lambda x: x['OriginCityName'] if not x['CheckStateMatch'] else x['Origin_City'], axis=1)


In [91]:
df[df['CheckStateMatch'] == False][['Origin', 'Origin_City', 'OriginState', 'PotentialState']].value_counts()

Origin  Origin_City     OriginState  PotentialState
DCA     Washington, DC  VA           DC                143090
IAD     Washington, DC  VA           DC                 75090
CVG     Cincinnati, OH  KY           OH                 41702
Name: count, dtype: int64

In [196]:
df['Origin_City'] = df['Origin_City'].str.replace(r'Cincinnati, OH', 'Hebron', regex=True)

In [197]:
df[df['CheckStateMatch'] == False][['Origin_City', 'OriginState']].value_counts()

Origin_City     OriginState
Washington, DC  VA             218180
Hebron          KY              41702
Name: count, dtype: int64

In [198]:
# Split the 'OriginCityName' into two columns but handle cases where there might not be a comma
df['Destination_City'] = df['DestCityName'].str.split(',').str[0]
df['PotentialState_D'] = df['DestCityName'].str.split(',').str[1]

# Fill NaNs in 'PotentialState' with an empty string or a placeholder if no comma was present
df['PotentialState_D'] = df['PotentialState_D'].fillna('')

# Trim any whitespace that might be around the state names
df['PotentialState_D'] = df['PotentialState_D'].str.strip()

# Proceed with your state comparison and adjustment
# This assumes you have an 'OriginState' column to compare with
df['CheckStateMatch_D'] = df['PotentialState_D'] == df['DestState']
df['Destination_City'] = df.apply(lambda x: x['DestCityName'] if not x['CheckStateMatch_D'] else x['Destination_City'], axis=1)


In [199]:
df['Destination_City'].head()

0    Charlotte
1    Charlotte
2    Charlotte
3    Charlotte
4    Charlotte
Name: Destination_City, dtype: object

In [200]:
df[df['CheckStateMatch'] == False][['Destination_City', 'DestState']].value_counts()
df['Destination_City'] = df['Destination_City'].str.replace(r'Cincinnati, OH', 'Hebron', regex=True)
df[df['CheckStateMatch_D'] == False][['Destination_City', 'DestState']].value_counts()

Destination_City  DestState
Washington, DC    VA           218077
Hebron            KY            41735
Name: count, dtype: int64

In [201]:
location_details_to_drop.extend(['OriginCityName', 'PotentialState', 'CheckStateMatch', 'DestCityName', 'PotentialState_D', 'CheckStateMatch_D'])

In [202]:
origin_states = df.groupby(['OriginState', 'OriginStateName', 'OriginStateFips', 'OriginWac']).size().reset_index()
origin_states.rename(columns={0:'Count'}, inplace=True)
origin_states.sort_values('OriginStateName').head()

,OriginState,OriginStateName,OriginStateFips,OriginWac,Count
1,AL,Alabama,1,51,36418
0,AK,Alaska,2,1,42728
3,AZ,Arizona,4,81,205794
2,AR,Arkansas,5,71,28750
4,CA,California,6,91,747100


In [203]:
dest_states = df.groupby(['DestState', 'DestStateName', 'DestStateFips', 'DestWac']).size().reset_index()
dest_states.rename(columns={0:'Count'}, inplace=True)
states_df = origin_states.merge(dest_states, left_on=['OriginStateFips', 'OriginWac'], right_on=['DestStateFips', 'DestWac'], suffixes=('_origin', '_dest'), how='outer')
states_df

,OriginState,OriginStateName,OriginStateFips,OriginWac,Count_origin,DestState,DestStateName,DestStateFips,DestWac,Count_dest
0,AK,Alaska,2,1,42728,AK,Alaska,2,1,42707
1,AL,Alabama,1,51,36418,AL,Alabama,1,51,36443
2,AR,Arkansas,5,71,28750,AR,Arkansas,5,71,28748
3,AZ,Arizona,4,81,205794,AZ,Arizona,4,81,205985
4,CA,California,6,91,747100,CA,California,6,91,747618
5,CO,Colorado,8,82,334351,CO,Colorado,8,82,333737
6,CT,Connecticut,9,11,22858,CT,Connecticut,9,11,22877
7,FL,Florida,12,33,612995,FL,Florida,12,33,612351
8,GA,Georgia,13,34,363670,GA,Georgia,13,34,363813
9,HI,Hawaii,15,2,132336,HI,Hawaii,15,2,132270


There don't appear to be any data entry issues with state abbreviations, state names or their FIPS codes. I will remove the FIPS and State Names columns and only keep the abbreviations. The FIPS and State Names will be preserved in a dictionary for use later if needed. 

In [204]:
states_dict = {row['OriginState']: (row['OriginStateName'], row['OriginStateFips'], row['OriginWac'])
               for index, row in origin_states.iterrows()}
states_dict

{'AK': ('Alaska', 2, 1),
 'AL': ('Alabama', 1, 51),
 'AR': ('Arkansas', 5, 71),
 'AZ': ('Arizona', 4, 81),
 'CA': ('California', 6, 91),
 'CO': ('Colorado', 8, 82),
 'CT': ('Connecticut', 9, 11),
 'FL': ('Florida', 12, 33),
 'GA': ('Georgia', 13, 34),
 'HI': ('Hawaii', 15, 2),
 'IA': ('Iowa', 19, 61),
 'ID': ('Idaho', 16, 83),
 'IL': ('Illinois', 17, 41),
 'IN': ('Indiana', 18, 42),
 'KS': ('Kansas', 20, 62),
 'KY': ('Kentucky', 21, 52),
 'LA': ('Louisiana', 22, 72),
 'MA': ('Massachusetts', 25, 13),
 'MD': ('Maryland', 24, 35),
 'ME': ('Maine', 23, 12),
 'MI': ('Michigan', 26, 43),
 'MN': ('Minnesota', 27, 63),
 'MO': ('Missouri', 29, 64),
 'MS': ('Mississippi', 28, 53),
 'MT': ('Montana', 30, 84),
 'NC': ('North Carolina', 37, 36),
 'ND': ('North Dakota', 38, 66),
 'NE': ('Nebraska', 31, 65),
 'NH': ('New Hampshire', 33, 14),
 'NJ': ('New Jersey', 34, 21),
 'NM': ('New Mexico', 35, 86),
 'NV': ('Nevada', 32, 85),
 'NY': ('New York', 36, 22),
 'OH': ('Ohio', 39, 44),
 'OK': ('Oklahoma

In [205]:
location_details_to_drop.extend(['OriginStateName', 'OriginStateFips', 'OriginWac', 'DestStateName', 'DestStateFips', 'DestWac'])
df = df.drop(columns=location_details_to_drop)

## Cleaning Departure and Arrival Performance Data

I will be able to apply data cleaning steps to the Depature-related columns. I will then need to separate out the cancelled flights to apply the changes to the Arrival-related columns. Cancelled flights are missing information about arrivals, so I need to preserve these NaN values for now.

In [206]:
df.head()

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,DOT_ID_Marketing_Airline,IATA_Code_Marketing_Airline,Flight_Number_Marketing_Airline,DOT_ID_Operating_Airline,IATA_Code_Operating_Airline,Tail_Number,OriginAirportID,OriginCityMarketID,Origin,OriginState,DestAirportID,DestCityMarketID,Dest,DestState,CRSDepTime,DepTime,DepDelay,DepDelayMinutes,DepDel15,DepartureDelayGroups,DepTimeBlk,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrTime,ArrDelay,ArrDelayMinutes,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,Cancelled,Diverted,CRSElapsedTime,ActualElapsedTime,AirTime,Flights,Distance,DistanceGroup,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,DivAirportLandings,Duplicate,FlightDate_date_only,Code_Share_Flight,Origin_City,Destination_City
0,2023,3,8,6,7,2023-08-06,20416,NK,2252,20416,NK,N978NK,12889,32211,LAS,NV,11057,31057,CLT,NC,2359,NaN,NaN,NaN,NaN,NaN,2300-2359,NaN,NaN,NaN,NaN,722,NaN,NaN,NaN,NaN,NaN,0700-0759,1.0,0.0,263.0,NaN,NaN,1.0,1916.0,8,NaN,NaN,NaN,NaN,NaN,0.0,N,2023-08-06,0,Las Vegas,Charlotte
1,2023,3,8,7,1,2023-08-07,20416,NK,2252,20416,NK,N974NK,12889,32211,LAS,NV,11057,31057,CLT,NC,2359,115.0,76.0,76.0,1.0,5.0,2300-2359,19.0,134.0,822.0,9.0,722,831.0,69.0,69.0,1.0,4.0,0700-0759,0.0,0.0,263.0,256.0,228.0,1.0,1916.0,8,3.0,0.0,1.0,0.0,65.0,0.0,N,2023-08-07,0,Las Vegas,Charlotte
2,2023,3,8,9,3,2023-08-09,20416,NK,2252,20416,NK,N519NK,12889,32211,LAS,NV,11057,31057,CLT,NC,2310,2259.0,-11.0,0.0,0.0,-1.0,2300-2359,14.0,2313.0,605.0,10.0,628,615.0,-13.0,0.0,0.0,-1.0,0600-0659,0.0,0.0,258.0,256.0,232.0,1.0,1916.0,8,NaN,NaN,NaN,NaN,NaN,0.0,N,2023-08-09,0,Las Vegas,Charlotte
3,2023,3,8,10,4,2023-08-10,20416,NK,2252,20416,NK,N532NK,12889,32211,LAS,NV,11057,31057,CLT,NC,2310,2302.0,-8.0,0.0,0.0,-1.0,2300-2359,12.0,2314.0,551.0,7.0,628,558.0,-30.0,0.0,0.0,-2.0,0600-0659,0.0,0.0,258.0,236.0,217.0,1.0,1916.0,8,NaN,NaN,NaN,NaN,NaN,0.0,N,2023-08-10,0,Las Vegas,Charlotte
4,2023,3,8,12,6,2023-08-12,20416,NK,2252,20416,NK,N529NK,12889,32211,LAS,NV,11057,31057,CLT,NC,2310,2314.0,4.0,4.0,0.0,0.0,2300-2359,18.0,2332.0,618.0,6.0,628,624.0,-4.0,0.0,0.0,-1.0,0600-0659,0.0,0.0,258.0,250.0,226.0,1.0,1916.0,8,NaN,NaN,NaN,NaN,NaN,0.0,N,2023-08-12,0,Las Vegas,Charlotte


In [207]:
df['CRSDepTime'].sort_values()

2805194       1
1822034       1
2395074       1
262265        1
2901124       1
           ... 
3830642    2359
2005316    2359
3830585    2359
3830174    2359
0          2359
Name: CRSDepTime, Length: 7260941, dtype: int64

Several time columns are currently represented as integers and floats instead of time objects. 

In [208]:
"""def convert_to_time(df, col):

    # conver time columns to integers, then strings and fill left spaces with 0
    df[col] = df[col].astype(int).astype(str).str.zfill(4)
    
    # insert a colon to create a time string format
    df[col] = df[col].str[:2] + ':' + df[col].str[2:]
    
    # Replace "24:00" with "00:00" to avoid conversion errors
    df[col] = df[col].replace('24:00', '00:00')

    # convert string to datetime time object
    df[col] = pd.to_datetime(df[col], format='%H:%M').dt.time

df_not_cancelled = df[~cancelled_flights].copy()

df_cancelled = df[cancelled_flights].copy()

# convert departure and arrival columns for not cancelled flights
columns_to_convert = ['CRSDepTime', 'DepTime', 'WheelsOff', 'WheelsOn', 'CRSArrTime', 'ArrTime']

for col in columns_to_convert:
    convert_to_time(df_not_cancelled, col)

# convert scheduled departure and arrival columns
columns_to_convert = ['CRSDepTime', 'CRSArrTime']

for col in columns_to_convert:
    convert_to_time(df_cancelled, col)    

# convert columns into integers since they represent boolean details
df_not_cancelled[['Cancelled', 'Diverted', 'DepDel15', 'ArrDel15']] = df_not_cancelled[['Cancelled', 'Diverted', 'DepDel15', 'ArrDel15']].astype(int)
# convert columns into integers since they represent boolean details
df_cancelled[['Cancelled', 'Diverted']] = df_cancelled[['Cancelled', 'Diverted']].astype(int)

# merge
df_merged = pd.concat([df_not_cancelled, df_cancelled])
df_merged"""

'def convert_to_time(df, col):\n\n    # conver time columns to integers, then strings and fill left spaces with 0\n    df[col] = df[col].astype(int).astype(str).str.zfill(4)\n    \n    # insert a colon to create a time string format\n    df[col] = df[col].str[:2] + \':\' + df[col].str[2:]\n    \n    # Replace "24:00" with "00:00" to avoid conversion errors\n    df[col] = df[col].replace(\'24:00\', \'00:00\')\n\n    # convert string to datetime time object\n    df[col] = pd.to_datetime(df[col], format=\'%H:%M\').dt.time\n\ndf_not_cancelled = df[~cancelled_flights].copy()\n\ndf_cancelled = df[cancelled_flights].copy()\n\n# convert departure and arrival columns for not cancelled flights\ncolumns_to_convert = [\'CRSDepTime\', \'DepTime\', \'WheelsOff\', \'WheelsOn\', \'CRSArrTime\', \'ArrTime\']\n\nfor col in columns_to_convert:\n    convert_to_time(df_not_cancelled, col)\n\n# convert scheduled departure and arrival columns\ncolumns_to_convert = [\'CRSDepTime\', \'CRSArrTime\']\n\nfor col 

The above code has been refactored to deal with flights that depart but return and are cancelled. I added an if / else clause in the `convert_to_time` function to handle the null values during the conversion. 

In [209]:
df_1 = df

In [210]:
def convert_to_time(df, col):
    # Apply the conversion only to non-NaN values
    df[col] = df[col].apply(lambda x: str(int(x)).zfill(4) if pd.notna(x) else x)
    
    # Insert a colon to create a time string format
    df[col] = df[col].apply(lambda x: x[:2] + ':' + x[2:] if pd.notna(x) else x)
    
    # Replace "24:00" with "00:00" to avoid conversion errors
    df[col] = df[col].replace('24:00', '00:00')
    
    # Convert string to datetime time object, but skip NaN values
    df[col] = pd.to_datetime(df[col], format='%H:%M', errors='coerce').dt.time


# convert scheduled departure and arrival columns
columns_to_convert = ['CRSDepTime', 'DepTime', 'WheelsOff', 'WheelsOn', 'CRSArrTime', 'ArrTime']

for col in columns_to_convert:
    convert_to_time(df, col)

In [211]:
cols_to_convert = ['Cancelled', 'Diverted']

for col in cols_to_convert:
    df[col] = df[col].apply(lambda x: int(x) if pd.notna(x) else x)

In [212]:
df[['Cancelled', 'Diverted', 'DepDel15', 'ArrDel15']].info()

<class 'pandas.core.frame.DataFrame'>
Index: 7260941 entries, 0 to 7278738
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   Cancelled  int64  
 1   Diverted   int64  
 2   DepDel15   float64
 3   ArrDel15   float64
dtypes: float64(2), int64(2)
memory usage: 277.0 MB


In [213]:
# convert columns into integers since they represent boolean details
#df_not_cancelled[['Cancelled', 'Diverted', 'DepDel15', 'ArrDel15']] = df_not_cancelled[['Cancelled', 'Diverted', 'DepDel15', 'ArrDel15']].astype(int)

In [214]:
# convert columns into integers since they represent boolean details
#df_cancelled[['Cancelled', 'Diverted']] = df_cancelled[['Cancelled', 'Diverted']].astype(int)

For the `DepDelay` vs `DepDelayMinutes` and `ArrDelay` vs `ArrDelayMinutes` columns, the only difference is that in the Minutes columns, early departures or arrivals are set to 0 instead of a negative number. This is redundant information so we can remove these columns.

In [215]:
# delete redundant columns
df_merged = df.drop(columns=['DepDelayMinutes', 'ArrDelayMinutes'])

In [216]:
df_merged.head()

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,DOT_ID_Marketing_Airline,IATA_Code_Marketing_Airline,Flight_Number_Marketing_Airline,DOT_ID_Operating_Airline,IATA_Code_Operating_Airline,Tail_Number,OriginAirportID,OriginCityMarketID,Origin,OriginState,DestAirportID,DestCityMarketID,Dest,DestState,CRSDepTime,DepTime,DepDelay,DepDel15,DepartureDelayGroups,DepTimeBlk,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrTime,ArrDelay,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,Cancelled,Diverted,CRSElapsedTime,ActualElapsedTime,AirTime,Flights,Distance,DistanceGroup,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,DivAirportLandings,Duplicate,FlightDate_date_only,Code_Share_Flight,Origin_City,Destination_City
0,2023,3,8,6,7,2023-08-06,20416,NK,2252,20416,NK,N978NK,12889,32211,LAS,NV,11057,31057,CLT,NC,23:59:00,NaT,NaN,NaN,NaN,2300-2359,NaN,NaT,NaT,NaN,07:22:00,NaT,NaN,NaN,NaN,0700-0759,1,0,263.0,NaN,NaN,1.0,1916.0,8,NaN,NaN,NaN,NaN,NaN,0.0,N,2023-08-06,0,Las Vegas,Charlotte
1,2023,3,8,7,1,2023-08-07,20416,NK,2252,20416,NK,N974NK,12889,32211,LAS,NV,11057,31057,CLT,NC,23:59:00,01:15:00,76.0,1.0,5.0,2300-2359,19.0,01:34:00,08:22:00,9.0,07:22:00,08:31:00,69.0,1.0,4.0,0700-0759,0,0,263.0,256.0,228.0,1.0,1916.0,8,3.0,0.0,1.0,0.0,65.0,0.0,N,2023-08-07,0,Las Vegas,Charlotte
2,2023,3,8,9,3,2023-08-09,20416,NK,2252,20416,NK,N519NK,12889,32211,LAS,NV,11057,31057,CLT,NC,23:10:00,22:59:00,-11.0,0.0,-1.0,2300-2359,14.0,23:13:00,06:05:00,10.0,06:28:00,06:15:00,-13.0,0.0,-1.0,0600-0659,0,0,258.0,256.0,232.0,1.0,1916.0,8,NaN,NaN,NaN,NaN,NaN,0.0,N,2023-08-09,0,Las Vegas,Charlotte
3,2023,3,8,10,4,2023-08-10,20416,NK,2252,20416,NK,N532NK,12889,32211,LAS,NV,11057,31057,CLT,NC,23:10:00,23:02:00,-8.0,0.0,-1.0,2300-2359,12.0,23:14:00,05:51:00,7.0,06:28:00,05:58:00,-30.0,0.0,-2.0,0600-0659,0,0,258.0,236.0,217.0,1.0,1916.0,8,NaN,NaN,NaN,NaN,NaN,0.0,N,2023-08-10,0,Las Vegas,Charlotte
4,2023,3,8,12,6,2023-08-12,20416,NK,2252,20416,NK,N529NK,12889,32211,LAS,NV,11057,31057,CLT,NC,23:10:00,23:14:00,4.0,0.0,0.0,2300-2359,18.0,23:32:00,06:18:00,6.0,06:28:00,06:24:00,-4.0,0.0,-1.0,0600-0659,0,0,258.0,250.0,226.0,1.0,1916.0,8,NaN,NaN,NaN,NaN,NaN,0.0,N,2023-08-12,0,Las Vegas,Charlotte


In [217]:
# Split the 'DepTimeBlk' into two columns to see if all blocks are 59 minutes long or are there some that are longer and shorter
df_merged['DeptTimeBlk_Start'] = df_merged['DepTimeBlk'].str.split('-').str[0].astype(int)
df_merged['DeptTimeBlk_End'] = df_merged['DepTimeBlk'].str.split('-').str[1].astype(int)

# Split the 'ArrTimeBlk' into two columns to see if all blocks are 59 minutes long or are there some that are longer and shorter
df_merged['ArrTimeBlk_Start'] = df_merged['ArrTimeBlk'].str.split('-').str[0].astype(int)
df_merged['ArrTimeBlk_End'] = df_merged['ArrTimeBlk'].str.split('-').str[1].astype(int)

In [218]:
df_merged['ArrivalBlock_Total'] = df_merged['ArrTimeBlk_End'] - df_merged['ArrTimeBlk_Start']
df_merged['DepartureBlock_Total'] = df_merged['DeptTimeBlk_End'] - df_merged['DeptTimeBlk_Start']

In [219]:
df_merged[['DepartureBlock_Total', 'ArrivalBlock_Total']].describe().apply(lambda x: x.apply('{0:.2f}'.format))

,DepartureBlock_Total,ArrivalBlock_Total
count,7260941.00,7260941.00
mean,74.18,72.08
std,85.69,79.72
min,59.00,59.00
25%,59.00,59.00
50%,59.00,59.00
75%,59.00,59.00
max,558.00,558.00


I'm curious to know why some flights have 558 minutes for their scheduled block. Let's take a look.

In [220]:
df_merged.sort_values('DepartureBlock_Total', ascending=False).head(10)

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,DOT_ID_Marketing_Airline,IATA_Code_Marketing_Airline,Flight_Number_Marketing_Airline,DOT_ID_Operating_Airline,IATA_Code_Operating_Airline,Tail_Number,OriginAirportID,OriginCityMarketID,Origin,OriginState,DestAirportID,DestCityMarketID,Dest,DestState,CRSDepTime,DepTime,DepDelay,DepDel15,DepartureDelayGroups,DepTimeBlk,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrTime,ArrDelay,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,Cancelled,Diverted,CRSElapsedTime,ActualElapsedTime,AirTime,Flights,Distance,DistanceGroup,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,DivAirportLandings,Duplicate,FlightDate_date_only,Code_Share_Flight,Origin_City,Destination_City,DeptTimeBlk_Start,DeptTimeBlk_End,ArrTimeBlk_Start,ArrTimeBlk_End,ArrivalBlock_Total,DepartureBlock_Total
4564220,2023,2,5,28,7,2023-05-28,19790,DL,1116,19790,DL,N681DA,15304,33195,TPA,FL,10397,30397,ATL,GA,05:20:00,05:13:00,-7.0,0.0,-1.0,0001-0559,12.0,05:25:00,06:26:00,5.0,06:50:00,06:31:00,-19.0,0.0,-2.0,0600-0659,0,0,90.0,78.0,61.0,1.0,406.0,2,NaN,NaN,NaN,NaN,NaN,0.0,N,2023-05-28,0,Tampa,Atlanta,1,559,600,659,59,558
1631850,2023,3,7,25,2,2023-07-25,19393,WN,3486,19393,WN,N8511K,13232,30977,MDW,IL,13796,32457,OAK,CA,05:25:00,05:28:00,3.0,0.0,0.0,0001-0559,9.0,05:37:00,07:30:00,7.0,07:50:00,07:37:00,-13.0,0.0,-1.0,0700-0759,0,0,265.0,249.0,233.0,1.0,1844.0,8,NaN,NaN,NaN,NaN,NaN,0.0,N,2023-07-25,0,Chicago,Oakland,1,559,700,759,59,558
1631812,2023,3,7,25,2,2023-07-25,19393,WN,3781,19393,WN,N8643A,13232,30977,MDW,IL,12953,31703,LGA,NY,05:20:00,05:17:00,-3.0,0.0,-1.0,0001-0559,10.0,05:27:00,08:03:00,7.0,08:25:00,08:10:00,-15.0,0.0,-1.0,0800-0859,0,0,125.0,113.0,96.0,1.0,725.0,3,NaN,NaN,NaN,NaN,NaN,0.0,N,2023-07-25,0,Chicago,New York,1,559,800,859,59,558
2971774,2023,2,6,9,5,2023-06-09,19393,WN,1653,19393,WN,N8560Z,13198,33198,MCI,MO,12191,31453,HOU,TX,05:20:00,05:24:00,4.0,0.0,0.0,0001-0559,13.0,05:37:00,07:08:00,6.0,07:20:00,07:14:00,-6.0,0.0,-1.0,0700-0759,0,0,120.0,110.0,91.0,1.0,666.0,3,NaN,NaN,NaN,NaN,NaN,0.0,N,2023-06-09,0,Kansas City,Houston,1,559,700,759,59,558
1631821,2023,3,7,25,2,2023-07-25,19393,WN,3773,19393,WN,N7841A,13232,30977,MDW,IL,13198,33198,MCI,MO,05:50:00,05:44:00,-6.0,0.0,-1.0,0001-0559,9.0,05:53:00,06:50:00,4.0,07:10:00,06:54:00,-16.0,0.0,-2.0,0700-0759,0,0,80.0,70.0,57.0,1.0,405.0,2,NaN,NaN,NaN,NaN,NaN,0.0,N,2023-07-25,0,Chicago,Kansas City,1,559,700,759,59,558
1631825,2023,3,7,25,2,2023-07-25,19393,WN,1865,19393,WN,N408WN,13232,30977,MDW,IL,13204,31454,MCO,FL,05:30:00,05:51:00,21.0,1.0,1.0,0001-0559,8.0,05:59:00,09:02:00,7.0,09:00:00,09:09:00,9.0,0.0,0.0,0900-0959,0,0,150.0,138.0,123.0,1.0,990.0,4,NaN,NaN,NaN,NaN,NaN,0.0,N,2023-07-25,0,Chicago,Orlando,1,559,900,959,59,558
2971759,2023,2,6,9,5,2023-06-09,19393,WN,1126,19393,WN,N8853Q,13198,33198,MCI,MO,11259,30194,DAL,TX,05:40:00,05:40:00,0.0,0.0,0.0,0001-0559,15.0,05:55:00,06:59:00,4.0,07:10:00,07:03:00,-7.0,0.0,-1.0,0700-0759,0,0,90.0,83.0,64.0,1.0,461.0,2,NaN,NaN,NaN,NaN,NaN,0.0,N,2023-06-09,0,Kansas City,Dallas,1,559,700,759,59,558
2971756,2023,2,6,9,5,2023-06-09,19393,WN,3352,19393,WN,N400WN,13198,33198,MCI,MO,10821,30852,BWI,MD,05:30:00,05:33:00,3.0,0.0,0.0,0001-0559,12.0,05:45:00,08:54:00,2.0,08:50:00,08:56:00,6.0,0.0,0.0,0800-0859,0,0,140.0,143.0,129.0,1.0,967.0,4,NaN,NaN,NaN,NaN,NaN,0.0,N,2023-06-09,0,Kansas City,Baltimore,1,559,800,859,59,558
1631846,2023,3,7,25,2,2023-07-25,19393,WN,3667,19393,WN,N244WN,13232,30977,MDW,IL,13495,33495,MSY,LA,05:45:00,05:39:00,-6.0,0.0,-1.0,0001-0559,10.0,05:49:00,07:33:00,4.0,08:00:00,07:37:00,-23.0,0.0,-2.0,0800-0859,0,0,135.0,118.0,104.0,1.0,825.0,4,NaN,NaN,NaN,NaN,NaN,0.0,N,2023-07-25,0,Chicago,New Orleans,1,559,800,859,59,558
2971743,2023,2,6,9,5,2023-06-09,19393,WN,1615,19393,WN,N8550Q,13158,33158,MAF,TX,12191,31453,HOU,TX,05:30:00,05:23:00,-7.0,0.0,-1.0,0001-0559,NaN,NaT,NaT,NaN,06:55:00,NaT,NaN,NaN,NaN,0600-0659,1,0,85.0,NaN,NaN,1.0,441.0,2,NaN,NaN,NaN,NaN,NaN,0.0,N,2023-06-09,0,Midland/Odes

In [221]:
from datetime import time

after_midnight_flights = df_merged[df_merged['CRSDepTime'] > time(0, 0)]
after_midnight_flights.sort_values('CRSDepTime').head()

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,DOT_ID_Marketing_Airline,IATA_Code_Marketing_Airline,Flight_Number_Marketing_Airline,DOT_ID_Operating_Airline,IATA_Code_Operating_Airline,Tail_Number,OriginAirportID,OriginCityMarketID,Origin,OriginState,DestAirportID,DestCityMarketID,Dest,DestState,CRSDepTime,DepTime,DepDelay,DepDel15,DepartureDelayGroups,DepTimeBlk,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrTime,ArrDelay,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,Cancelled,Diverted,CRSElapsedTime,ActualElapsedTime,AirTime,Flights,Distance,DistanceGroup,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,DivAirportLandings,Duplicate,FlightDate_date_only,Code_Share_Flight,Origin_City,Destination_City,DeptTimeBlk_Start,DeptTimeBlk_End,ArrTimeBlk_Start,ArrTimeBlk_End,ArrivalBlock_Total,DepartureBlock_Total
2805194,2023,2,6,9,5,2023-06-09,19790,DL,304,19790,DL,N665DN,12889,32211,LAS,NV,10397,30397,ATL,GA,00:01:00,00:00:00,-1.0,0.0,-1.0,0001-0559,12.0,00:12:00,06:36:00,10.0,06:56:00,06:46:00,-10.0,0.0,-1.0,0600-0659,0,0,235.0,226.0,204.0,1.0,1747.0,7,NaN,NaN,NaN,NaN,NaN,0.0,N,2023-06-09,0,Las Vegas,Atlanta,1,559,600,659,59,558
1822034,2023,3,7,3,1,2023-07-03,20436,F9,1102,20436,F9,N343FR,12889,32211,LAS,NV,13204,31454,MCO,FL,00:01:00,00:17:00,16.0,1.0,1.0,0001-0559,19.0,00:36:00,07:31:00,14.0,07:33:00,07:45:00,12.0,0.0,0.0,0700-0759,0,0,272.0,268.0,235.0,1.0,2039.0,9,NaN,NaN,NaN,NaN,NaN,0.0,N,2023-07-03,0,Las Vegas,Orlando,1,559,700,759,59,558
2395074,2023,4,12,7,4,2023-12-07,20436,F9,3234,20436,F9,N317FR,12889,32211,LAS,NV,11042,30647,CLE,OH,00:01:00,23:57:00,-4.0,0.0,-1.0,0001-0559,15.0,00:12:00,06:45:00,8.0,06:55:00,06:53:00,-2.0,0.0,-1.0,0600-0659,0,0,234.0,236.0,213.0,1.0,1824.0,8,NaN,NaN,NaN,NaN,NaN,0.0,N,2023-12-07,0,Las Vegas,Cleveland,1,559,600,659,59,558
262265,2023,3,8,7,1,2023-08-07,20436,F9,4880,20436,F9,N344FR,12889,32211,LAS,NV,13303,32467,MIA,FL,00:01:00,01:24:00,83.0,1.0,5.0,0001-0559,13.0,01:37:00,08:43:00,11.0,07:54:00,08:54:00,60.0,1.0,4.0,0700-0759,0,0,293.0,270.0,246.0,1.0,2174.0,9,4.0,0.0,0.0,0.0,56.0,0.0,N,2023-08-07,0,Las Vegas,Miami,1,559,700,759,59,558
2901124,2023,2,6,23,5,2023-06-23,20436,F9,1102,20436,F9,NaN,12889,32211,LAS,NV,13204,31454,MCO,FL,00:01:00,NaT,NaN,NaN,NaN,0001-0559,NaN,NaT,NaT,NaN,07:33:00,NaT,NaN,NaN,NaN,0700-0759,1,0,272.0,NaN,NaN,1.0,2039.0,9,NaN,NaN,NaN,NaN,NaN,0.0,N,2023-06-23,0,Las Vegas,Orlando,1,559,700,759,59,558


Flights that depart or arrive after midnight and before 6AM have 558 minutes blocked for their departure/arrival time. All other flights only have 59 minutes for their scheduled block. This is likely due to allow for logistics and scheduling for airport operations for these overnight/really early flights. It may be helpful to categorize flights as overnight departure or overnight arrival events compared to all other flights. 

In [222]:
overnight_departures = (df_merged['DepartureBlock_Total'] > 60)
overnight_arrivals = (df_merged['ArrivalBlock_Total'] > 60)

In [223]:
df_merged['Overnight_Depature'] = overnight_departures.astype(int)
df_merged['Overnight_Arrival'] = overnight_arrivals.astype(int)

In [224]:
df_merged.sort_values('CRSDepTime').head()

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,DOT_ID_Marketing_Airline,IATA_Code_Marketing_Airline,Flight_Number_Marketing_Airline,DOT_ID_Operating_Airline,IATA_Code_Operating_Airline,Tail_Number,OriginAirportID,OriginCityMarketID,Origin,OriginState,DestAirportID,DestCityMarketID,Dest,DestState,CRSDepTime,DepTime,DepDelay,DepDel15,DepartureDelayGroups,DepTimeBlk,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrTime,ArrDelay,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,Cancelled,Diverted,CRSElapsedTime,ActualElapsedTime,AirTime,Flights,Distance,DistanceGroup,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,DivAirportLandings,Duplicate,FlightDate_date_only,Code_Share_Flight,Origin_City,Destination_City,DeptTimeBlk_Start,DeptTimeBlk_End,ArrTimeBlk_Start,ArrTimeBlk_End,ArrivalBlock_Total,DepartureBlock_Total,Overnight_Depature,Overnight_Arrival
2805194,2023,2,6,9,5,2023-06-09,19790,DL,304,19790,DL,N665DN,12889,32211,LAS,NV,10397,30397,ATL,GA,00:01:00,00:00:00,-1.0,0.0,-1.0,0001-0559,12.0,00:12:00,06:36:00,10.0,06:56:00,06:46:00,-10.0,0.0,-1.0,0600-0659,0,0,235.0,226.0,204.0,1.0,1747.0,7,NaN,NaN,NaN,NaN,NaN,0.0,N,2023-06-09,0,Las Vegas,Atlanta,1,559,600,659,59,558,1,0
1822034,2023,3,7,3,1,2023-07-03,20436,F9,1102,20436,F9,N343FR,12889,32211,LAS,NV,13204,31454,MCO,FL,00:01:00,00:17:00,16.0,1.0,1.0,0001-0559,19.0,00:36:00,07:31:00,14.0,07:33:00,07:45:00,12.0,0.0,0.0,0700-0759,0,0,272.0,268.0,235.0,1.0,2039.0,9,NaN,NaN,NaN,NaN,NaN,0.0,N,2023-07-03,0,Las Vegas,Orlando,1,559,700,759,59,558,1,0
2395074,2023,4,12,7,4,2023-12-07,20436,F9,3234,20436,F9,N317FR,12889,32211,LAS,NV,11042,30647,CLE,OH,00:01:00,23:57:00,-4.0,0.0,-1.0,0001-0559,15.0,00:12:00,06:45:00,8.0,06:55:00,06:53:00,-2.0,0.0,-1.0,0600-0659,0,0,234.0,236.0,213.0,1.0,1824.0,8,NaN,NaN,NaN,NaN,NaN,0.0,N,2023-12-07,0,Las Vegas,Cleveland,1,559,600,659,59,558,1,0
262265,2023,3,8,7,1,2023-08-07,20436,F9,4880,20436,F9,N344FR,12889,32211,LAS,NV,13303,32467,MIA,FL,00:01:00,01:24:00,83.0,1.0,5.0,0001-0559,13.0,01:37:00,08:43:00,11.0,07:54:00,08:54:00,60.0,1.0,4.0,0700-0759,0,0,293.0,270.0,246.0,1.0,2174.0,9,4.0,0.0,0.0,0.0,56.0,0.0,N,2023-08-07,0,Las Vegas,Miami,1,559,700,759,59,558,1,0
2901124,2023,2,6,23,5,2023-06-23,20436,F9,1102,20436,F9,NaN,12889,32211,LAS,NV,13204,31454,MCO,FL,00:01:00,NaT,NaN,NaN,NaN,0001-0559,NaN,NaT,NaT,NaN,07:33:00,NaT,NaN,NaN,NaN,0700-0759,1,0,272.0,NaN,NaN,1.0,2039.0,9,NaN,NaN,NaN,NaN,NaN,0.0,N,2023-06-23,0,Las Vegas,Orlando,1,559,700,759,59,558,1,0


In [225]:
flight_perf_to_drop = ['DepTimeBlk', 'ArrTimeBlk', 'DeptTimeBlk_Start', 'DeptTimeBlk_End', 'ArrTimeBlk_Start', 'ArrTimeBlk_End', 'ArrivalBlock_Total', 'DepartureBlock_Total']
df_merged = df_merged.drop(columns=flight_perf_to_drop)

In [226]:
df_merged.head()

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,DOT_ID_Marketing_Airline,IATA_Code_Marketing_Airline,Flight_Number_Marketing_Airline,DOT_ID_Operating_Airline,IATA_Code_Operating_Airline,Tail_Number,OriginAirportID,OriginCityMarketID,Origin,OriginState,DestAirportID,DestCityMarketID,Dest,DestState,CRSDepTime,DepTime,DepDelay,DepDel15,DepartureDelayGroups,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrTime,ArrDelay,ArrDel15,ArrivalDelayGroups,Cancelled,Diverted,CRSElapsedTime,ActualElapsedTime,AirTime,Flights,Distance,DistanceGroup,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,DivAirportLandings,Duplicate,FlightDate_date_only,Code_Share_Flight,Origin_City,Destination_City,Overnight_Depature,Overnight_Arrival
0,2023,3,8,6,7,2023-08-06,20416,NK,2252,20416,NK,N978NK,12889,32211,LAS,NV,11057,31057,CLT,NC,23:59:00,NaT,NaN,NaN,NaN,NaN,NaT,NaT,NaN,07:22:00,NaT,NaN,NaN,NaN,1,0,263.0,NaN,NaN,1.0,1916.0,8,NaN,NaN,NaN,NaN,NaN,0.0,N,2023-08-06,0,Las Vegas,Charlotte,0,0
1,2023,3,8,7,1,2023-08-07,20416,NK,2252,20416,NK,N974NK,12889,32211,LAS,NV,11057,31057,CLT,NC,23:59:00,01:15:00,76.0,1.0,5.0,19.0,01:34:00,08:22:00,9.0,07:22:00,08:31:00,69.0,1.0,4.0,0,0,263.0,256.0,228.0,1.0,1916.0,8,3.0,0.0,1.0,0.0,65.0,0.0,N,2023-08-07,0,Las Vegas,Charlotte,0,0
2,2023,3,8,9,3,2023-08-09,20416,NK,2252,20416,NK,N519NK,12889,32211,LAS,NV,11057,31057,CLT,NC,23:10:00,22:59:00,-11.0,0.0,-1.0,14.0,23:13:00,06:05:00,10.0,06:28:00,06:15:00,-13.0,0.0,-1.0,0,0,258.0,256.0,232.0,1.0,1916.0,8,NaN,NaN,NaN,NaN,NaN,0.0,N,2023-08-09,0,Las Vegas,Charlotte,0,0
3,2023,3,8,10,4,2023-08-10,20416,NK,2252,20416,NK,N532NK,12889,32211,LAS,NV,11057,31057,CLT,NC,23:10:00,23:02:00,-8.0,0.0,-1.0,12.0,23:14:00,05:51:00,7.0,06:28:00,05:58:00,-30.0,0.0,-2.0,0,0,258.0,236.0,217.0,1.0,1916.0,8,NaN,NaN,NaN,NaN,NaN,0.0,N,2023-08-10,0,Las Vegas,Charlotte,0,0
4,2023,3,8,12,6,2023-08-12,20416,NK,2252,20416,NK,N529NK,12889,32211,LAS,NV,11057,31057,CLT,NC,23:10:00,23:14:00,4.0,0.0,0.0,18.0,23:32:00,06:18:00,6.0,06:28:00,06:24:00,-4.0,0.0,-1.0,0,0,258.0,250.0,226.0,1.0,1916.0,8,NaN,NaN,NaN,NaN,NaN,0.0,N,2023-08-12,0,Las Vegas,Charlotte,0,0


## Check for duplicates

In [227]:
df_merged.duplicated().sum()

0

In [228]:
df_merged['Duplicate'].value_counts()

Duplicate
N    7260941
Name: count, dtype: int64

In [229]:
df_merged = df_merged.drop(columns='Duplicate')

## Check column values

In [230]:
df_merged['Flights'].value_counts()

Flights
1.0    7260941
Name: count, dtype: int64

The `Flights` column only contains the same value across all rows and is therefore not providing any new information.

In [231]:
df_merged.drop(columns='Flights', inplace=True)

In [232]:
# check unique values for day of
np.sort(df_merged['DayOfWeek'].unique())

array([1, 2, 3, 4, 5, 6, 7])

There are only 7 unique values for day of week, as I would expect. According to the data dictionary, 1 = Monday and 7 = Sunday. They reserve 9 for unknown day of week, however, there are no occurences of an unknown day of week in this dataset. 

In [233]:
df_merged.head()

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,DOT_ID_Marketing_Airline,IATA_Code_Marketing_Airline,Flight_Number_Marketing_Airline,DOT_ID_Operating_Airline,IATA_Code_Operating_Airline,Tail_Number,OriginAirportID,OriginCityMarketID,Origin,OriginState,DestAirportID,DestCityMarketID,Dest,DestState,CRSDepTime,DepTime,DepDelay,DepDel15,DepartureDelayGroups,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrTime,ArrDelay,ArrDel15,ArrivalDelayGroups,Cancelled,Diverted,CRSElapsedTime,ActualElapsedTime,AirTime,Distance,DistanceGroup,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,DivAirportLandings,FlightDate_date_only,Code_Share_Flight,Origin_City,Destination_City,Overnight_Depature,Overnight_Arrival
0,2023,3,8,6,7,2023-08-06,20416,NK,2252,20416,NK,N978NK,12889,32211,LAS,NV,11057,31057,CLT,NC,23:59:00,NaT,NaN,NaN,NaN,NaN,NaT,NaT,NaN,07:22:00,NaT,NaN,NaN,NaN,1,0,263.0,NaN,NaN,1916.0,8,NaN,NaN,NaN,NaN,NaN,0.0,2023-08-06,0,Las Vegas,Charlotte,0,0
1,2023,3,8,7,1,2023-08-07,20416,NK,2252,20416,NK,N974NK,12889,32211,LAS,NV,11057,31057,CLT,NC,23:59:00,01:15:00,76.0,1.0,5.0,19.0,01:34:00,08:22:00,9.0,07:22:00,08:31:00,69.0,1.0,4.0,0,0,263.0,256.0,228.0,1916.0,8,3.0,0.0,1.0,0.0,65.0,0.0,2023-08-07,0,Las Vegas,Charlotte,0,0
2,2023,3,8,9,3,2023-08-09,20416,NK,2252,20416,NK,N519NK,12889,32211,LAS,NV,11057,31057,CLT,NC,23:10:00,22:59:00,-11.0,0.0,-1.0,14.0,23:13:00,06:05:00,10.0,06:28:00,06:15:00,-13.0,0.0,-1.0,0,0,258.0,256.0,232.0,1916.0,8,NaN,NaN,NaN,NaN,NaN,0.0,2023-08-09,0,Las Vegas,Charlotte,0,0
3,2023,3,8,10,4,2023-08-10,20416,NK,2252,20416,NK,N532NK,12889,32211,LAS,NV,11057,31057,CLT,NC,23:10:00,23:02:00,-8.0,0.0,-1.0,12.0,23:14:00,05:51:00,7.0,06:28:00,05:58:00,-30.0,0.0,-2.0,0,0,258.0,236.0,217.0,1916.0,8,NaN,NaN,NaN,NaN,NaN,0.0,2023-08-10,0,Las Vegas,Charlotte,0,0
4,2023,3,8,12,6,2023-08-12,20416,NK,2252,20416,NK,N529NK,12889,32211,LAS,NV,11057,31057,CLT,NC,23:10:00,23:14:00,4.0,0.0,0.0,18.0,23:32:00,06:18:00,6.0,06:28:00,06:24:00,-4.0,0.0,-1.0,0,0,258.0,250.0,226.0,1916.0,8,NaN,NaN,NaN,NaN,NaN,0.0,2023-08-12,0,Las Vegas,Charlotte,0,0


# Review IATA Codes Data

I'll inpsect the IATA Codes dataset and fill in the Marketing and Operating Airline codes with the airline names for readability.

In [234]:
df_codes.head()

,Code,Description
0,19031,Mackey International Inc.: MAC
1,19032,Munz Northern Airlines Inc.: XY
2,19033,Cochise Airlines Inc.: COC
3,19034,Golden Gate Airlines Inc.: GSA
4,19035,Aeromech Inc.: RZZ


In [235]:
df_codes.shape

(1737, 2)

In [236]:
df_codes['Airline_Name'] = df_codes['Description'].str.split(':').str[0].str.strip()
df_codes['IATA_Code'] = df_codes['Description'].str.split(':').str[1].str.strip()

In [237]:
df_codes.head()

,Code,Description,Airline_Name,IATA_Code
0,19031,Mackey International Inc.: MAC,Mackey International Inc.,MAC
1,19032,Munz Northern Airlines Inc.: XY,Munz Northern Airlines Inc.,XY
2,19033,Cochise Airlines Inc.: COC,Cochise Airlines Inc.,COC
3,19034,Golden Gate Airlines Inc.: GSA,Golden Gate Airlines Inc.,GSA
4,19035,Aeromech Inc.: RZZ,Aeromech Inc.,RZZ


In [238]:
duplicate_iata = df_codes[df_codes['IATA_Code'].duplicated(keep=False)]
sorted_duplicates = duplicate_iata.sort_values(by='IATA_Code')
sorted_duplicates

,Code,Description,Airline_Name,IATA_Code


## Replace IATA Codes in flights df with airline names

In [239]:
# create dictionary of codes and airlines
iata_dict = df_codes.set_index('IATA_Code')['Airline_Name'].to_dict()
iata_dict

{'MAC': 'Mackey International Inc.',
 'XY': 'Munz Northern Airlines Inc.',
 'COC': 'Cochise Airlines Inc.',
 'GSA': 'Golden Gate Airlines Inc.',
 'RZZ': 'Aeromech Inc.',
 'GLW': 'Golden West Airlines Co.',
 'PRN': 'Puerto Rico Intl Airlines',
 'STZ': 'Air America Inc.',
 'SWT': 'Swift Aire Lines Inc.',
 'TSF': 'American Central Airlines',
 'VEZ': 'Valdez Airlines',
 'WEB': 'Southeast Alaska Airlines',
 'AAR': 'Altair Airlines Inc.',
 'CHI': 'Chitina Air Service',
 'MRC': 'Marco Island Airways Inc.',
 'OHZ': 'Caribbean Air Services Inc.',
 'PRO': 'Sundance Airlines',
 'SAI': 'Seair Alaska Airlines Inc.',
 'SLZ': 'Southeast Airlines Inc.',
 'AAZ': 'Alaska Aeronautical Indust.',
 'IMP': 'Imperial Airlines Inc.',
 'TWA': 'Trans Western Airlines Utah',
 'WRT': 'Wright Airlines Inc.',
 'COL': 'Presidential Express',
 'MVA': 'Mississippi Valley Airlines',
 'CHL': 'Channel Flying Inc.',
 'JCZ': 'Rocky Mountain Airways Inc.',
 'MIS': 'Midstate Airlines Inc.',
 'STG': 'Sedalia Marshall Boonvl St

In [240]:
df_merged['Airline_Mkt'] = df_merged['IATA_Code_Marketing_Airline'].map(iata_dict)
df_merged['Airline_Ops'] = df_merged['IATA_Code_Operating_Airline'].map(iata_dict)

In [241]:
df_merged[['Airline_Mkt']].isna().sum()

Airline_Mkt    0
dtype: int64

In [242]:
df_merged[['Airline_Ops']].isna().sum()

Airline_Ops    0
dtype: int64

## Merge IATA Code data to flight performance data

In [243]:
# drop IATA codes because we have a dictionary with these details if needed
df_merged = df_merged.drop(columns=['IATA_Code_Marketing_Airline', 'IATA_Code_Operating_Airline'])

In [244]:
df_merged.head()

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,DOT_ID_Marketing_Airline,Flight_Number_Marketing_Airline,DOT_ID_Operating_Airline,Tail_Number,OriginAirportID,OriginCityMarketID,Origin,OriginState,DestAirportID,DestCityMarketID,Dest,DestState,CRSDepTime,DepTime,DepDelay,DepDel15,DepartureDelayGroups,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrTime,ArrDelay,ArrDel15,ArrivalDelayGroups,Cancelled,Diverted,CRSElapsedTime,ActualElapsedTime,AirTime,Distance,DistanceGroup,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,DivAirportLandings,FlightDate_date_only,Code_Share_Flight,Origin_City,Destination_City,Overnight_Depature,Overnight_Arrival,Airline_Mkt,Airline_Ops
0,2023,3,8,6,7,2023-08-06,20416,2252,20416,N978NK,12889,32211,LAS,NV,11057,31057,CLT,NC,23:59:00,NaT,NaN,NaN,NaN,NaN,NaT,NaT,NaN,07:22:00,NaT,NaN,NaN,NaN,1,0,263.0,NaN,NaN,1916.0,8,NaN,NaN,NaN,NaN,NaN,0.0,2023-08-06,0,Las Vegas,Charlotte,0,0,Spirit Air Lines,Spirit Air Lines
1,2023,3,8,7,1,2023-08-07,20416,2252,20416,N974NK,12889,32211,LAS,NV,11057,31057,CLT,NC,23:59:00,01:15:00,76.0,1.0,5.0,19.0,01:34:00,08:22:00,9.0,07:22:00,08:31:00,69.0,1.0,4.0,0,0,263.0,256.0,228.0,1916.0,8,3.0,0.0,1.0,0.0,65.0,0.0,2023-08-07,0,Las Vegas,Charlotte,0,0,Spirit Air Lines,Spirit Air Lines
2,2023,3,8,9,3,2023-08-09,20416,2252,20416,N519NK,12889,32211,LAS,NV,11057,31057,CLT,NC,23:10:00,22:59:00,-11.0,0.0,-1.0,14.0,23:13:00,06:05:00,10.0,06:28:00,06:15:00,-13.0,0.0,-1.0,0,0,258.0,256.0,232.0,1916.0,8,NaN,NaN,NaN,NaN,NaN,0.0,2023-08-09,0,Las Vegas,Charlotte,0,0,Spirit Air Lines,Spirit Air Lines
3,2023,3,8,10,4,2023-08-10,20416,2252,20416,N532NK,12889,32211,LAS,NV,11057,31057,CLT,NC,23:10:00,23:02:00,-8.0,0.0,-1.0,12.0,23:14:00,05:51:00,7.0,06:28:00,05:58:00,-30.0,0.0,-2.0,0,0,258.0,236.0,217.0,1916.0,8,NaN,NaN,NaN,NaN,NaN,0.0,2023-08-10,0,Las Vegas,Charlotte,0,0,Spirit Air Lines,Spirit Air Lines
4,2023,3,8,12,6,2023-08-12,20416,2252,20416,N529NK,12889,32211,LAS,NV,11057,31057,CLT,NC,23:10:00,23:14:00,4.0,0.0,0.0,18.0,23:32:00,06:18:00,6.0,06:28:00,06:24:00,-4.0,0.0,-1.0,0,0,258.0,250.0,226.0,1916.0,8,NaN,NaN,NaN,NaN,NaN,0.0,2023-08-12,0,Las Vegas,Charlotte,0,0,Spirit Air Lines,Spirit Air Lines


# Import geographical data for airports

https://www.transtats.bts.gov/DL_SelectFields.aspx?gnoyr_VQ=FLL&QO_fu146_anzr=N8vn6v10%20f722146%20gnoyr5

In [5]:
df_airports = pd.read_csv(DATA_PATH + '/raw/T_MASTER_CORD.csv')

In [6]:
df_airports.head()

,AIRPORT_SEQ_ID,AIRPORT_ID,AIRPORT,DISPLAY_AIRPORT_NAME,DISPLAY_AIRPORT_CITY_NAME_FULL,AIRPORT_WAC,AIRPORT_COUNTRY_NAME,AIRPORT_COUNTRY_CODE_ISO,AIRPORT_STATE_NAME,AIRPORT_STATE_CODE,AIRPORT_STATE_FIPS,CITY_MARKET_ID,DISPLAY_CITY_MARKET_NAME_FULL,CITY_MARKET_WAC,LAT_DEGREES,LAT_HEMISPHERE,LAT_MINUTES,LAT_SECONDS,LATITUDE,LON_DEGREES,LON_HEMISPHERE,LON_MINUTES,LON_SECONDS,LONGITUDE,AIRPORT_START_DATE,AIRPORT_THRU_DATE,AIRPORT_IS_CLOSED,AIRPORT_IS_LATEST
0,1000101,10001,01A,Afognak Lake Airport,"Afognak Lake, AK",1,United States,US,Alaska,AK,2.0,30001,"Afognak Lake, AK",1,58.0,N,6.0,34.0,58.109444,152.0,W,54.0,24.0,-152.906667,7/1/2007 12:00:00 AM,NaN,0,1
1,1000301,10003,03A,Bear Creek Mining Strip,"Granite Mountain, AK",1,United States,US,Alaska,AK,2.0,30003,"Granite Mountain, AK",1,65.0,N,32.0,53.0,65.548056,161.0,W,4.0,18.0,-161.071667,7/1/2007 12:00:00 AM,NaN,0,1
2,1000401,10004,04A,Lik Mining Camp,"Lik, AK",1,United States,US,Alaska,AK,2.0,30004,"Lik, AK",1,68.0,N,5.0,0.0,68.083333,163.0,W,10.0,0.0,-163.166667,7/1/2007 12:00:00 AM,NaN,0,1
3,1000501,10005,05A,Little Squaw Airport,"Little Squaw, AK",1,United States,US,Alaska,AK,2.0,30005,"Little Squaw, AK",1,67.0,N,34.0,12.0,67.570000,148.0,W,11.0,2.0,-148.183889,8/1/2007 12:00:00 AM,NaN,0,1
4,1000601,10006,06A,Kizhuyak Bay,"Kizhuyak, AK",1,United States,US,Alaska,AK,2.0,30006,"Kizhuyak, AK",1,57.0,N,44.0,43.0,57.745278,152.0,W,52.0,58.0,-152.882778,10/1/2007 12:00:00 AM,NaN,0,1


In [7]:
df_airports['AIRPORT_COUNTRY_NAME'].value_counts()

AIRPORT_COUNTRY_NAME
United States     7293
Canada             935
Australia          581
Brazil             458
France             449
                  ... 
Carriacou            1
Wake Island          1
Kyrgyzstan           1
Norfolk Island       1
Macau                1
Name: count, Length: 256, dtype: int64

In [8]:
df_airports.shape

(19197, 28)

We only need US airports for the purpose of this project. I will drop all rows for international airports.

In [9]:
df_airports = df_airports[df_airports['AIRPORT_COUNTRY_NAME'] == 'United States']

In [10]:
df_airports.columns

Index(['AIRPORT_SEQ_ID', 'AIRPORT_ID', 'AIRPORT', 'DISPLAY_AIRPORT_NAME',
       'DISPLAY_AIRPORT_CITY_NAME_FULL', 'AIRPORT_WAC', 'AIRPORT_COUNTRY_NAME',
       'AIRPORT_COUNTRY_CODE_ISO', 'AIRPORT_STATE_NAME', 'AIRPORT_STATE_CODE',
       'AIRPORT_STATE_FIPS', 'CITY_MARKET_ID', 'DISPLAY_CITY_MARKET_NAME_FULL',
       'CITY_MARKET_WAC', 'LAT_DEGREES', 'LAT_HEMISPHERE', 'LAT_MINUTES',
       'LAT_SECONDS', 'LATITUDE', 'LON_DEGREES', 'LON_HEMISPHERE',
       'LON_MINUTES', 'LON_SECONDS', 'LONGITUDE', 'AIRPORT_START_DATE',
       'AIRPORT_THRU_DATE', 'AIRPORT_IS_CLOSED', 'AIRPORT_IS_LATEST'],
      dtype='object')

In [11]:
df_airports.drop(columns=[
    'AIRPORT_COUNTRY_NAME', 
    'AIRPORT_COUNTRY_CODE_ISO', 
    'LAT_DEGREES', 
    'LAT_HEMISPHERE', 
    'LAT_MINUTES',
    'LAT_SECONDS', 
    'LON_DEGREES', 
    'LON_HEMISPHERE',
    'LON_MINUTES', 
    'LON_SECONDS'
],
inplace=True)

In [12]:
df_airports.head()

,AIRPORT_SEQ_ID,AIRPORT_ID,AIRPORT,DISPLAY_AIRPORT_NAME,DISPLAY_AIRPORT_CITY_NAME_FULL,AIRPORT_WAC,AIRPORT_STATE_NAME,AIRPORT_STATE_CODE,AIRPORT_STATE_FIPS,CITY_MARKET_ID,DISPLAY_CITY_MARKET_NAME_FULL,CITY_MARKET_WAC,LATITUDE,LONGITUDE,AIRPORT_START_DATE,AIRPORT_THRU_DATE,AIRPORT_IS_CLOSED,AIRPORT_IS_LATEST
0,1000101,10001,01A,Afognak Lake Airport,"Afognak Lake, AK",1,Alaska,AK,2.0,30001,"Afognak Lake, AK",1,58.109444,-152.906667,7/1/2007 12:00:00 AM,NaN,0,1
1,1000301,10003,03A,Bear Creek Mining Strip,"Granite Mountain, AK",1,Alaska,AK,2.0,30003,"Granite Mountain, AK",1,65.548056,-161.071667,7/1/2007 12:00:00 AM,NaN,0,1
2,1000401,10004,04A,Lik Mining Camp,"Lik, AK",1,Alaska,AK,2.0,30004,"Lik, AK",1,68.083333,-163.166667,7/1/2007 12:00:00 AM,NaN,0,1
3,1000501,10005,05A,Little Squaw Airport,"Little Squaw, AK",1,Alaska,AK,2.0,30005,"Little Squaw, AK",1,67.570000,-148.183889,8/1/2007 12:00:00 AM,NaN,0,1
4,1000601,10006,06A,Kizhuyak Bay,"Kizhuyak, AK",1,Alaska,AK,2.0,30006,"Kizhuyak, AK",1,57.745278,-152.882778,10/1/2007 12:00:00 AM,NaN,0,1


In [93]:
df_airports['AIRPORT'].nunique()

2964

In [94]:
len(df_airports['AIRPORT'])

7293

In [98]:
df_airports.sort_values('AIRPORT', ascending=False).head(20)

,AIRPORT_SEQ_ID,AIRPORT_ID,AIRPORT,DISPLAY_AIRPORT_NAME,DISPLAY_AIRPORT_CITY_NAME_FULL,AIRPORT_WAC,AIRPORT_STATE_NAME,AIRPORT_STATE_CODE,AIRPORT_STATE_FIPS,CITY_MARKET_ID,DISPLAY_CITY_MARKET_NAME_FULL,CITY_MARKET_WAC,LATITUDE,LONGITUDE,AIRPORT_START_DATE,AIRPORT_THRU_DATE,AIRPORT_IS_CLOSED,AIRPORT_IS_LATEST
19196,9999901,99999,ZZZ,Unknown Point in Alaska,Unknown Point in Alaska,1,Alaska,AK,2.0,99999,Unknown Point in Alaska,1,NaN,NaN,1/1/1950 12:00:00 AM,NaN,0,1
18104,1636101,16361,ZZV,Zanesville Municipal,"Zanesville, OH",44,Ohio,OH,39.0,36361,"Zanesville, OH",44,39.942778,-81.892500,1/1/1950 12:00:00 AM,6/30/2011 12:00:00 AM,0,0
18108,1636105,16361,ZZV,Zanesville Municipal,"Zanesville, OH",44,Ohio,OH,39.0,36361,"Zanesville, OH",44,39.944444,-81.891944,12/1/2017 12:00:00 AM,9/30/2022 12:00:00 AM,0,0
18107,1636104,16361,ZZV,Zanesville Municipal,"Zanesville, OH",44,Ohio,OH,39.0,36361,"Zanesville, OH",44,39.944444,-81.892222,11/1/2017 12:00:00 AM,11/30/2017 12:00:00 AM,0,0
18106,1636103,16361,ZZV,Zanesville Municipal,"Zanesville, OH",44,Ohio,OH,39.0,36361,"Zanesville, OH",44,39.944444,-81.891944,10/1/2017 12:00:00 AM,10/31/2017 12:00:00 AM,0,0
18105,1636102,16361,ZZV,Zanesville Municipal,"Zanesville, OH",44,Ohio,OH,39.0,36361,"Zanesville, OH",44,39.944444,-81.892222,7/1/2011 12:00:00 AM,9/30/2017 12:00:00 AM,0,0
18109,1636106,16361,ZZV,Zanesville Municipal,"Zanesville, OH",44,Ohio,OH,39.0,36361,"Zanesville, OH",44,39.944444,-81.892222,10/1/2022 12:00:00 AM,NaN,0,1
18097,1635802,16358,ZXZ,Waterville Airport,"Waterville, WA",93,Washington,WA,53.0,36358,"Waterville, WA",93,47.656111,-120.056389,7/1/2011 12:00:00 AM,NaN,0,1
18096,1635801,16358,ZXZ,Waterville Airport,"Waterville, WA",93,Washington,WA,53.0,36358,"Waterville, WA",93,47.654167,-120.054167,3/1/1990 12:00:00 AM,6/30/2011 12:00:00 AM,0,0
18095,1635703,16357,ZXY,Blake Field,"Delta, CO",82,Colorado,CO,8.0,36357,"Delta, CO",82,38.785556,-108.061944,7/1/2019 12:00:00 AM,NaN,0,1


In [104]:
current_airports = (df_airports['AIRPORT_IS_LATEST'] == 1) & (df_airports['AIRPORT_IS_CLOSED'] == 0)
df_airports[current_airports]['AIRPORT'].nunique()

2804

In [105]:
airports_clean = df_airports[current_airports]

### Adding timezone information for each airport

In [ ]:
from timezonefinder import TimezoneFinder


In [106]:
airports_clean.to_csv(DATA_PATH + '/interim/airports.csv', index=False)

#### Save updated flights df that has timezone corrections and UTC conversion

In [ ]:
df_merged.to_csv(DATA_PATH + '/interim/2023-performance-data-clean.csv', index=False)
df_merged.info()

# Import Aircraft Registration Data

https://www.faa.gov/licenses_certificates/aircraft_certification/aircraft_registry/releasable_aircraft_download

In [247]:
aircraft_registration = pd.read_csv(DATA_PATH + '/raw/ReleasableAircraft-2023/MASTER.txt')

/var/folders/b2/nkpmrw0s42q_zm32rwshljyr0000gn/T/ipykernel_76448/2070737829.py:1: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  aircraft_registration = pd.read_csv(DATA_PATH + '/raw/ReleasableAircraft-2023/MASTER.txt')


In [248]:
aircraft_ref = pd.read_csv(DATA_PATH + '/raw/ReleasableAircraft-2023/ACFTREF.txt')

In [249]:
engine_ref = pd.read_csv(DATA_PATH + '/raw/ReleasableAircraft-2023/ENGINE.txt')

In [250]:
aircraft_registration.head()

,N-NUMBER,SERIAL NUMBER,MFR MDL CODE,ENG MFR MDL,YEAR MFR,TYPE REGISTRANT,NAME,STREET,STREET2,CITY,STATE,ZIP CODE,REGION,COUNTY,COUNTRY,LAST ACTION DATE,CERT ISSUE DATE,CERTIFICATION,TYPE AIRCRAFT,TYPE ENGINE,STATUS CODE,MODE S CODE,FRACT OWNER,AIR WORTH DATE,OTHER NAMES(1),OTHER NAMES(2),OTHER NAMES(3),OTHER NAMES(4),OTHER NAMES(5),EXPIRATION DATE,UNIQUE ID,KIT MFR,KIT MODEL,MODE S CODE HEX,Unnamed: 34
0,1,680-0519,2076811,52041,2014,7,TENAX AEROSPACE LLC ...,400 W PARKWAY PL STE 201,,RIDGELAND,MS,391576005,2,089,US,20230823,20211130,1T,5,5,V,50000001,,20140325,...,...,...,...,...,20281130,1141371,,,A00001,NaN
1,100,5334,7100510,17003,1940,1,BENE MARY D ...,PO BOX 329,,KETCHUM,OK,743490329,2,097,US,20230122,20050506,1,4,1,V,50002263,,19540430,...,...,...,...,...,20270430,600060,,,A004B3,NaN
2,10001,A28,9601202,67007,1928,1,STOOS ROBERT A ...,PO BOX 1056,,LAKELAND,FL,338021056,7,105,US,20230718,20190227,1,4,1,V,50003446,,,...,...,...,...,...,20290228,432072,,,A00726,NaN
3,10004,T18208245,2072738,,,7,ETOS AIR LLC ...,PO BOX 288,,NEW LONDON,TX,756820288,2,401,US,20230722,20130312,,4,1,V,50003451,,,...,...,...,...,...,20290331,102879,,,A00729,NaN
4,10006,BG-72,1152020,17026,1955,1,COUTCHES ROBERT HERCULES DBA ...,550 AIRWAY BLVD,,LIVERMORE,CA,945519533,4,001,US,20230421,19980826,1U,4,1,V,50003453,,19710909,AERO FLIGHT AVIATION ...,...,...,...,...,20280229,480110,,,A0072B,NaN


In [251]:
aircraft_registration.shape

(293465, 35)

In [252]:
aircraft_registration.columns

Index(['N-NUMBER', 'SERIAL NUMBER', 'MFR MDL CODE', 'ENG MFR MDL', 'YEAR MFR',
       'TYPE REGISTRANT', 'NAME', 'STREET', 'STREET2', 'CITY', 'STATE',
       'ZIP CODE', 'REGION', 'COUNTY', 'COUNTRY', 'LAST ACTION DATE',
       'CERT ISSUE DATE', 'CERTIFICATION', 'TYPE AIRCRAFT', 'TYPE ENGINE',
       'STATUS CODE', 'MODE S CODE', 'FRACT OWNER', 'AIR WORTH DATE',
       'OTHER NAMES(1)', 'OTHER NAMES(2)', 'OTHER NAMES(3)', 'OTHER NAMES(4)',
       'OTHER NAMES(5)', 'EXPIRATION DATE', 'UNIQUE ID', 'KIT MFR',
       ' KIT MODEL', 'MODE S CODE HEX', 'Unnamed: 34'],
      dtype='object')

In [253]:
aircraft_registration.drop(columns=[
    'FRACT OWNER',
    'OTHER NAMES(1)', 
    'OTHER NAMES(2)', 
    'OTHER NAMES(3)', 
    'OTHER NAMES(4)',
    'OTHER NAMES(5)',
    'KIT MFR',
    ' KIT MODEL',
    'Unnamed: 34'], 
    inplace=True)

I'll filter the aircraft registration file for tail numbers in the monthly performance data since we don't need data about any other aircrafts from the FAA.

In [254]:
# check for number of aircrafts on FAA list for commercial flights
df_merged.Tail_Number.nunique()

6326

In [255]:
df_merged.Tail_Number.head()

0    N978NK
1    N974NK
2    N519NK
3    N532NK
4    N529NK
Name: Tail_Number, dtype: object

In [256]:
df_merged.Tail_Number.isna().sum()

19648

Just over 19,000 flights are missing Tail Numbers. 

In [257]:
# remove 'N' from Tail Number to compare to aircraft registrations
df_merged['Tail_Number'] = df['Tail_Number'].apply(lambda x: x[1:] if isinstance(x, str) and x.startswith('N') else x)
df_merged.Tail_Number.head()

0    978NK
1    974NK
2    519NK
3    532NK
4    529NK
Name: Tail_Number, dtype: object

In [258]:
tail_numbers = df_merged.Tail_Number

In [259]:
aircraft_registration = aircraft_registration[aircraft_registration['N-NUMBER'].isin(tail_numbers)]

In [260]:
aircraft_registration.shape

(6273, 26)

In [261]:
6326-6273

53

The 2023 aircraft registration is missing registrations for 53 planes. These might be in older registrations. I will download the 2022 data to compare. 

In [262]:
aircraft_registration.head()

,N-NUMBER,SERIAL NUMBER,MFR MDL CODE,ENG MFR MDL,YEAR MFR,TYPE REGISTRANT,NAME,STREET,STREET2,CITY,STATE,ZIP CODE,REGION,COUNTY,COUNTRY,LAST ACTION DATE,CERT ISSUE DATE,CERTIFICATION,TYPE AIRCRAFT,TYPE ENGINE,STATUS CODE,MODE S CODE,AIR WORTH DATE,EXPIRATION DATE,UNIQUE ID,MODE S CODE HEX
645,10156,145786,3260214,54582,2004,3,UNITED AIRLINES INC ...,233 S WACKER DR,,CHICAGO,IL,606067147,C,031,US,20230804,20190531,1T,5,5,V,50005621,20040215,20290531,90729,A00B91
807,101DQ,9409,3940005,13101,2020,3,DELTA AIR LINES INC ...,1775 M H JACKSON SERVICE RD,DEPT 595 AIRCRAFT REGISTRATIONS,ATLANTA,GA,30354,7,121,US,20231007,20200213,1T,5,5,V,50004305,20200213,20300228,1312557,A008C5
810,101DU,50020,1400012,52315,2018,3,BANK OF UTAH TRUSTEE ...,50 S 200 E STE 110,,SALT LAKE CITY,UT,841111617,S,035,US,20230906,20200808,1T,5,5,V,50004311,20181025,20270831,1285810,A008C9
866,101HQ,17000156,3260121,30061,2007,3,REPUBLIC AIRWAYS INC ...,8909 PURDUE RD STE 300,,INDIANAPOLIS,IN,462683152,C,097,US,20230818,20070227,1T,5,5,V,50004451,20070228,20290731,328579,A00929
938,101NN,5834,3930402,34601,2013,3,WILMINGTON TRUST CO TRUSTEE ...,1100 N MARKET ST,,WILMINGTON,DE,198901605,1,003,US,20230421,20210222,1T,5,5,V,50004644,20131122,20280229,1119973,A009A4


In [263]:
mask = df_merged['Tail_Number'].isin(aircraft_registration['N-NUMBER'])
missing_registration = df_merged[~mask]

In [264]:
missing_registration.head()

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,DOT_ID_Marketing_Airline,Flight_Number_Marketing_Airline,DOT_ID_Operating_Airline,Tail_Number,OriginAirportID,OriginCityMarketID,Origin,OriginState,DestAirportID,DestCityMarketID,Dest,DestState,CRSDepTime,DepTime,DepDelay,DepDel15,DepartureDelayGroups,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrTime,ArrDelay,ArrDel15,ArrivalDelayGroups,Cancelled,Diverted,CRSElapsedTime,ActualElapsedTime,AirTime,Distance,DistanceGroup,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,DivAirportLandings,FlightDate_date_only,Code_Share_Flight,Origin_City,Destination_City,Overnight_Depature,Overnight_Arrival,Airline_Mkt,Airline_Ops
11878,2023,3,8,30,3,2023-08-30,19805,5016,20397,NaN,12451,31136,JAX,FL,11278,30852,DCA,VA,12:02:00,NaT,NaN,NaN,NaN,NaN,NaT,NaT,NaN,13:58:00,NaT,NaN,NaN,NaN,1,0,116.0,NaN,NaN,634.0,3,NaN,NaN,NaN,NaN,NaN,0.0,2023-08-30,1,Jacksonville,"Washington, DC",0,0,American Airlines Inc.,PSA Airlines Inc.
12397,2023,3,8,8,2,2023-08-08,19805,5036,20397,NaN,13931,33667,ORF,VA,11057,31057,CLT,NC,06:35:00,NaT,NaN,NaN,NaN,NaN,NaT,NaT,NaN,08:02:00,NaT,NaN,NaN,NaN,1,0,87.0,NaN,NaN,290.0,2,NaN,NaN,NaN,NaN,NaN,0.0,2023-08-08,1,Norfolk,Charlotte,0,0,American Airlines Inc.,PSA Airlines Inc.
12898,2023,3,8,31,4,2023-08-31,19805,5047,20397,NaN,14685,34685,SAV,GA,11298,30194,DFW,TX,07:26:00,NaT,NaN,NaN,NaN,NaN,NaT,NaT,NaN,09:21:00,NaT,NaN,NaN,NaN,1,0,175.0,NaN,NaN,925.0,4,NaN,NaN,NaN,NaN,NaN,0.0,2023-08-31,1,Savannah,Dallas/Fort Worth,0,0,American Airlines Inc.,PSA Airlines Inc.
12953,2023,3,8,31,4,2023-08-31,19805,5049,20397,NaN,14685,34685,SAV,GA,11057,31057,CLT,NC,07:41:00,NaT,NaN,NaN,NaN,NaN,NaT,NaT,NaN,09:04:00,NaT,NaN,NaN,NaN,1,0,83.0,NaN,NaN,213.0,1,NaN,NaN,NaN,NaN,NaN,0.0,2023-08-31,1,Savannah,Charlotte,0,0,American Airlines Inc.,PSA Airlines Inc.
13344,2023,3,8,30,3,2023-08-30,19805,5059,20397,NaN,14100,34100,PHL,PA,12451,31136,JAX,FL,09:07:00,NaT,NaN,NaN,NaN,NaN,NaT,NaT,NaN,11:28:00,NaT,NaN,NaN,NaN,1,0,141.0,NaN,NaN,742.0,3,NaN,NaN,NaN,NaN,NaN,0.0,2023-08-30,1,Philadelphia,Jacksonville,0,0,American Airlines Inc.,PSA Airlines Inc.


In [265]:
df_ac_reg_2022 = pd.read_csv(DATA_PATH + '/raw/ReleasableAircraft-2022/MASTER.txt')

/var/folders/b2/nkpmrw0s42q_zm32rwshljyr0000gn/T/ipykernel_76448/938479805.py:1: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ac_reg_2022 = pd.read_csv(DATA_PATH + '/raw/ReleasableAircraft-2022/MASTER.txt')


In [266]:
aircraft_registration_2022 = df_ac_reg_2022[df_ac_reg_2022['N-NUMBER'].isin(missing_registration['Tail_Number'])]
aircraft_registration_2022.shape

(22, 35)

In [267]:
df_ac_reg_2021 = pd.read_csv(DATA_PATH + '/raw/ReleasableAircraft-2021/MASTER.txt')

/var/folders/b2/nkpmrw0s42q_zm32rwshljyr0000gn/T/ipykernel_76448/1670319963.py:1: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ac_reg_2021 = pd.read_csv(DATA_PATH + '/raw/ReleasableAircraft-2021/MASTER.txt')


In [268]:
aircraft_registration_2021 = df_ac_reg_2021[df_ac_reg_2021['N-NUMBER'].isin(missing_registration['Tail_Number'])]
aircraft_registration_2021.shape

(22, 35)

In [269]:
aircraft_registration_2021.merge(aircraft_registration_2022, on='N-NUMBER').shape

(22, 69)

2021 and 2022 aircraft registrations contain the same aircrafts that are missing from the 2023 df. I will add these. 

In [270]:
aircraft_registration_2022.drop(columns=[
    'FRACT OWNER',
    'OTHER NAMES(1)', 
    'OTHER NAMES(2)', 
    'OTHER NAMES(3)', 
    'OTHER NAMES(4)',
    'OTHER NAMES(5)',
    'KIT MFR',
    ' KIT MODEL',
    'Unnamed: 34'], 
    inplace=True)

/var/folders/b2/nkpmrw0s42q_zm32rwshljyr0000gn/T/ipykernel_76448/1062023511.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aircraft_registration_2022.drop(columns=[


In [271]:
aircraft_registration = pd.concat([aircraft_registration, aircraft_registration_2022], axis=0)

In [272]:
aircraft_registration.shape

(6295, 26)

In [273]:
aircraft_registration.head(2)

,N-NUMBER,SERIAL NUMBER,MFR MDL CODE,ENG MFR MDL,YEAR MFR,TYPE REGISTRANT,NAME,STREET,STREET2,CITY,STATE,ZIP CODE,REGION,COUNTY,COUNTRY,LAST ACTION DATE,CERT ISSUE DATE,CERTIFICATION,TYPE AIRCRAFT,TYPE ENGINE,STATUS CODE,MODE S CODE,AIR WORTH DATE,EXPIRATION DATE,UNIQUE ID,MODE S CODE HEX
645,10156,145786,3260214,54582,2004,3,UNITED AIRLINES INC ...,233 S WACKER DR,,CHICAGO,IL,606067147,C,031,US,20230804,20190531,1T,5,5,V,50005621,20040215,20290531,90729,A00B91
807,101DQ,9409,3940005,13101,2020,3,DELTA AIR LINES INC ...,1775 M H JACKSON SERVICE RD,DEPT 595 AIRCRAFT REGISTRATIONS,ATLANTA,GA,30354,7,121,US,20231007,20200213,1T,5,5,V,50004305,20200213,20300228,1312557,A008C5


In [274]:
aircraft_ref.head()

,CODE,MFR,MODEL,TYPE-ACFT,TYPE-ENG,AC-CAT,BUILD-CERT-IND,NO-ENG,NO-SEATS,AC-WEIGHT,SPEED,TC-DATA-SHEET,TC-DATA-HOLDER,Unnamed: 13
0,0020901,AAR AIRLIFT GROUP INC,UH-60A,6,3,1,0,2,15,CLASS 3,0,,...,NaN
1,0030109,EXLINE ACE-C,ACE-C,4,1,1,1,1,1,CLASS 1,82,,...,NaN
2,003010D,DELEBAUGH,P,4,1,1,1,1,1,CLASS 1,82,,...,NaN
3,003010H,DAL PORTO,BABY ACE D,4,1,1,1,1,1,CLASS 1,82,,...,NaN
4,003010P,DUNN,BABY ACE,4,1,1,1,1,1,CLASS 1,82,,...,NaN


In [275]:
aircraft_ref.drop(columns=['TC-DATA-SHEET', 'TC-DATA-HOLDER', 'Unnamed: 13'], inplace=True)

In [276]:
engine_ref.head()

,CODE,MFR,MODEL,TYPE,HORSEPOWER,THRUST,Unnamed: 6
0,0,NONE,NONE,0,0,0,NaN
1,401,A.C.E.,HIDR MARK III,1,95,0,NaN
2,402,A.C.E.,UPRI MARK III,1,100,0,NaN
3,450,AEROMOMENT,AM13 SERIES,8,100,0,NaN
4,452,AEROMOMENT,AM15 SERIES,8,117,0,NaN


In [277]:
engine_ref.drop(columns='Unnamed: 6', inplace=True)

In [278]:
aircraft_registration_merged = aircraft_registration.merge(aircraft_ref, left_on='MFR MDL CODE', right_on='CODE', how='left')

In [279]:
aircraft_registration_merged['ENG MFR MDL'] = aircraft_registration_merged['ENG MFR MDL'].replace('     ', 0).astype(int)
aircraft_registration_merged = aircraft_registration_merged.merge(engine_ref, left_on='ENG MFR MDL', right_on='CODE', how='left', suffixes=['_aircraft', '_engine'])

In [280]:
aircraft_registration_merged.head()

,N-NUMBER,SERIAL NUMBER,MFR MDL CODE,ENG MFR MDL,YEAR MFR,TYPE REGISTRANT,NAME,STREET,STREET2,CITY,STATE,ZIP CODE,REGION,COUNTY,COUNTRY,LAST ACTION DATE,CERT ISSUE DATE,CERTIFICATION,TYPE AIRCRAFT,TYPE ENGINE,STATUS CODE,MODE S CODE,AIR WORTH DATE,EXPIRATION DATE,UNIQUE ID,MODE S CODE HEX,CODE_aircraft,MFR_aircraft,MODEL_aircraft,TYPE-ACFT,TYPE-ENG,AC-CAT,BUILD-CERT-IND,NO-ENG,NO-SEATS,AC-WEIGHT,SPEED,CODE_engine,MFR_engine,MODEL_engine,TYPE,HORSEPOWER,THRUST
0,10156,145786,3260214,54582,2004,3,UNITED AIRLINES INC ...,233 S WACKER DR,,CHICAGO,IL,606067147,C,031,US,20230804,20190531,1T,5,5,V,50005621,20040215,20290531,90729,A00B91,3260214,EMBRAER,EMB-145XR,5,5,1,0,2,55,CLASS 2,0,54582,ROLLS-ROYC,AE3007 SER,5,0,6442
1,101DQ,9409,3940005,13101,2020,3,DELTA AIR LINES INC ...,1775 M H JACKSON SERVICE RD,DEPT 595 AIRCRAFT REGISTRATIONS,ATLANTA,GA,30354,7,121,US,20231007,20200213,1T,5,5,V,50004305,20200213,20300228,1312557,A008C5,3940005,AIRBUS,A321-211,5,5,1,0,2,199,CLASS 3,0,13101,CFM INTL,CFM56-5B3/3,5,0,32000
2,101DU,50020,1400012,52315,2018,3,BANK OF UTAH TRUSTEE ...,50 S 200 E STE 110,,SALT LAKE CITY,UT,841111617,S,035,US,20230906,20200808,1T,5,5,V,50004311,20181025,20270831,1285810,A008C9,1400012,C SERIES AIRCRAFT LTD PTNRSP,BD-500-1A10,5,5,1,0,2,133,CLASS 3,0,52315,P & W,PW1519G,5,0,19775
3,101HQ,17000156,3260121,30061,2007,3,REPUBLIC AIRWAYS INC ...,8909 PURDUE RD STE 300,,INDIANAPOLIS,IN,462683152,C,097,US,20230818,20070227,1T,5,5,V,50004451,20070228,20290731,328579,A00929,3260121,EMBRAER-EMPRESA BRASILEIRA DE,ERJ 170-200 LR,5,5,1,0,2,80,CLASS 3,0,30061,GE,CF34-8E5,5,0,14510
4,101NN,5834,3930402,34601,2013,3,WILMINGTON TRUST CO TRUSTEE ...,1100 N MARKET ST,,WILMINGTON,DE,198901605,1,003,US,20230421,20210222,1T,5,5,V,50004644,20131122,20280229,1119973,A009A4,3930402,AIRBUS INDUSTRIE,A321-231,5,5,1,0,2,379,CLASS 3,0,34601,IAE,V2500SERIES,4,0,25000


In [281]:
aircraft_registration_merged.to_csv(DATA_PATH + '/interim/aircraft_registration.csv', index=False)